In [ ]:
pip install transformers torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 96.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load a fine-tuned T5 model for question generation
model_name = "valhalla/t5-small-qg-hl"  # Use "t5-base-qg-hl" for better results
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

def generate_quiz_question(context):
    """Generate a multiple-choice question with a structured prompt."""
    input_text = f"Generate a properly structured multiple-choice question based on the following statement: {context}"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids

    # Generate output from the model
    output_ids = model.generate(input_ids, max_length=50, do_sample=True)
    question = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return question


# Example usage
context = "Newton's first law states that an object will remain at rest or in uniform motion unless acted upon by an external force."
quiz_question = generate_quiz_question(context)

print("Generated Question:", quiz_question)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


pytorch_model.bin:   0%|          | 0.00/242M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

Generated Question: A single-choice question that Newton says that objects will remain at rest or in uniform motion is not acted upon or controlled by external force that is unenseparable in the first law without additional intervention by a force.


In [ ]:
import random

# Function to generate quiz question with multiple-choice answers
def generate_mcq(context):
    question = generate_quiz_question(context)

    # Generate incorrect options (Placeholder for now)
    options = [
        "It explains how objects attract each other",
        "It states that energy is always conserved",
        "It describes motion in a curved path",
        "It states that an object remains at rest unless acted upon"
    ]
    random.shuffle(options)

    return {
        "question": question,
        "options": options,
        "answer": options[0]  # Placeholder answer
    }

# Example quiz from Newton's Law
quiz = generate_mcq("Newton's first law states that an object will remain at rest or in uniform motion unless acted upon by an external force.")
print(quiz)


{'question': 'What are the two theories of the different types of arguments that Newton proposed?', 'options': ['It states that energy is always conserved', 'It explains how objects attract each other', 'It describes motion in a curved path', 'It states that an object remains at rest unless acted upon'], 'answer': 'It states that energy is always conserved'}


In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer
import torch

# Load BERT QA Model
qa_model_name = "deepset/bert-base-cased-squad2"
qa_tokenizer = AutoTokenizer.from_pretrained(qa_model_name)
qa_model = AutoModelForQuestionAnswering.from_pretrained(qa_model_name)

def get_correct_answer(question, options, context):
    """Uses BERT to find the best matching answer."""
    scores = []

    for option in options:
        inputs = qa_tokenizer(question, option, return_tensors="pt")
        outputs = qa_model(**inputs)
        score = torch.softmax(outputs.start_logits, dim=1).max().item()
        scores.append((option, score))

    # Sort options by relevance score
    scores.sort(key=lambda x: x[1], reverse=True)
    return scores[0][0]  # Return best-scoring option

# Example question & answer selection
context = "Newton's first law states that an object will remain at rest or in uniform motion unless acted upon by an external force."
question = generate_quiz_question(context)
options = [
    "It explains how objects attract each other",
    "It states that energy is always conserved",
    "It describes motion in a curved path",
    "It states that an object remains at rest unless acted upon"
]

correct_answer = get_correct_answer(question, options, context)

quiz = {
    "question": question,
    "options": options,
    "answer": correct_answer
}

print(quiz)


tokenizer_config.json:   0%|          | 0.00/152 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at deepset/bert-base-cased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'question': 'It is common to a theory on how a person will be at rest or rotate, is it?', 'options': ['It explains how objects attract each other', 'It states that energy is always conserved', 'It describes motion in a curved path', 'It states that an object remains at rest unless acted upon'], 'answer': 'It states that an object remains at rest unless acted upon'}


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AutoModelForQuestionAnswering
import torch
import random

# Load models
t5_model_name = "valhalla/t5-small-qg-hl"
t5_tokenizer = AutoTokenizer.from_pretrained(t5_model_name)
t5_model = AutoModelForSeq2SeqLM.from_pretrained(t5_model_name)

qa_model_name = "deepset/bert-base-cased-squad2"
qa_tokenizer = AutoTokenizer.from_pretrained(qa_model_name)
qa_model = AutoModelForQuestionAnswering.from_pretrained(qa_model_name)

# Function to generate a proper quiz question
def generate_quiz_question(context):
    input_text = f"Generate a properly structured multiple-choice question based on this statement: {context}"
    input_ids = t5_tokenizer(input_text, return_tensors="pt").input_ids

    output_ids = t5_model.generate(input_ids, max_length=50, do_sample=True)
    question = t5_tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return question

# Function to get correct answer using BERT
def get_correct_answer(question, options, context):
    scores = []

    for option in options:
        inputs = qa_tokenizer(question, option, return_tensors="pt")
        outputs = qa_model(**inputs)

        start_score = torch.softmax(outputs.start_logits, dim=1).max().item()
        end_score = torch.softmax(outputs.end_logits, dim=1).max().item()
        total_score = (start_score + end_score) / 2  # More accurate scoring

        scores.append((option, total_score))

    scores.sort(key=lambda x: x[1], reverse=True)
    return scores[0][0]  # Return best answer

# Function to generate an MCQ with correct answer
def generate_mcq(context):
    question = generate_quiz_question(context)

    options = [
        "It explains how objects attract each other",
        "It states that energy is always conserved",
        "It describes motion in a curved path",
        "It states that an object remains at rest unless acted upon"
    ]
    random.shuffle(options)

    correct_answer = get_correct_answer(question, options, context)

    return {
        "question": question,
        "options": options,
        "answer": correct_answer
    }

# Example: Generate quiz for Newton's Law
context = "Newton's first law states that an object will remain at rest or in uniform motion unless acted upon by an external force."
quiz = generate_mcq(context)
print(quiz)


Some weights of the model checkpoint at deepset/bert-base-cased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'question': 'The first law states that an object will remain at rest or in uniform motion unless made possible by the external force?', 'options': ['It states that an object remains at rest unless acted upon', 'It explains how objects attract each other', 'It states that energy is always conserved', 'It describes motion in a curved path'], 'answer': 'It states that energy is always conserved'}


Llama for Quest

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load a fine-tuned T5 model for question generation
model_name = "valhalla/t5-small-qg-hl"  # Use "t5-base-qg-hl" for better results
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

def generate_quiz_question(context):
    """Generate a multiple-choice question with a structured prompt."""
    input_text = f"Generate a properly structured multiple-choice question based on the following statement: {context}"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids

    # Generate output from the model
    output_ids = model.generate(input_ids, max_length=50, do_sample=True)
    question = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return question


# Example usage
context = "Newton's first law states that an object will remain at rest or in uniform motion unless acted upon by an external force."
quiz_question = generate_quiz_question(context)

print("Generated Question:", quiz_question)


Generated Question: What is the name of the law in Newton?


In [ ]:
pip install sentence-transformers


In [ ]:
from sentence_transformers import SentenceTransformer, util

# Load SBERT for answer validation
sbert_model = SentenceTransformer('all-MiniLM-L6-v2')

# Function to get the best answer
def get_correct_answer_sbert(question, options):
    """Use SBERT to select the most relevant answer."""
    question_embedding = sbert_model.encode(question, convert_to_tensor=True)
    option_embeddings = sbert_model.encode(options, convert_to_tensor=True)

    # Compute similarity scores
    similarity_scores = util.pytorch_cos_sim(question_embedding, option_embeddings)[0]

    # Select the best answer
    best_answer_idx = similarity_scores.argmax().item()

    return options[best_answer_idx]


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
import random

# Function to generate full MCQ quiz
def generate_mcq(context):
    question = generate_quiz_question(context)

    options = [
        "It explains how objects attract each other",
        "It states that energy is always conserved",
        "It describes motion in a curved path",
        "It states that an object remains at rest unless acted upon"
    ]
    random.shuffle(options)

    correct_answer = get_correct_answer_sbert(question, options)

    return {
        "question": question,
        "options": options,
        "answer": correct_answer
    }

# Test the full MCQ generation
context = "Newton's first law states that an object will remain at rest or in uniform motion unless acted upon by an external force."
quiz = generate_mcq(context)
print(quiz)


{'question': 'What statement states that an object will remain at rest or in uniform motion without an external force?', 'options': ['It explains how objects attract each other', 'It states that energy is always conserved', 'It states that an object remains at rest unless acted upon', 'It describes motion in a curved path'], 'answer': 'It states that an object remains at rest unless acted upon'}


In [ ]:
import random
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from sentence_transformers import SentenceTransformer, util

# Load T5 model for quiz generation
model_name = "valhalla/t5-base-qg-hl"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Load SBERT for answer validation
sbert_model = SentenceTransformer('all-MiniLM-L6-v2')

# Function to generate quiz question

def generate_quiz_question(context):
    """Generate a multiple-choice question with a structured prompt."""
    input_text = f"Generate a properly structured multiple-choice question based on the following statement: {context}"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids

    # Generate output from the model
    output_ids = model.generate(input_ids, max_length=50, do_sample=True)
    question = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return question


# Function to get best answer using context matching
def get_correct_answer_with_context(question, options, context):
    """Selects the most contextually relevant answer."""
    scores = []

    for option in options:
        score = util.pytorch_cos_sim(
            sbert_model.encode(context, convert_to_tensor=True),
            sbert_model.encode(option, convert_to_tensor=True)
        )[0].item()
        scores.append((option, score))

    # Sort and return the highest-scoring answer
    scores.sort(key=lambda x: x[1], reverse=True)
    return scores[0][0]

# Function to generate a full MCQ
def generate_mcq(context):
    question = generate_quiz_question(context)

    options = [
        "It explains how objects attract each other",
        "It states that energy is always conserved",
        "It describes motion in a curved path",
        "It states that an object remains at rest unless acted upon"
    ]
    random.shuffle(options)

    correct_answer = get_correct_answer_with_context(question, options, context)

    return {
        "question": question,
        "options": options,
        "answer": correct_answer
    }

# Example quiz for Newton's Law
context = "Newton's first law states that an object will remain at rest or in uniform motion unless acted upon by an external force."
quiz = generate_mcq(context)
print(quiz)


tokenizer_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/15.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

{'question': 'What is the base on which you can form a choice?', 'options': ['It explains how objects attract each other', 'It states that energy is always conserved', 'It describes motion in a curved path', 'It states that an object remains at rest unless acted upon'], 'answer': 'It states that an object remains at rest unless acted upon'}


In [ ]:
pip install transformers datasets torch accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.6 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

# Load SQuAD (General Knowledge & Science)
squad_dataset = load_dataset("squad")

# Load MathQA (Mathematics)
mathqa_dataset = load_dataset("math_qa")

# Load AI2 Science Questions (Physics, Chemistry, Biology)
sciq_dataset = load_dataset("ai2_arc", "ARC-Easy")

print("SQuAD Sample:", squad_dataset["train"][0])
print("MathQA Sample:", mathqa_dataset["train"][0])
print("Science Sample:", sciq_dataset["train"][0])


README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

...

validation-00000-of-00001.parquet:   0%|          | 0.00/86.1k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2251 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2376 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/570 [00:00<?, ? examples/s]

SQuAD Sample: {'id': '5733be284776f41900661182', 'title': 'University_of_Notre_Dame', 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.', 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?', 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}
MathQA Sample: {'Problem': "the banker '

In [ ]:
def convert_squad_to_quiz(dataset):
    quiz_data = []

    for data in dataset["train"]:
        context = data["context"]
        question = data["question"]

        quiz_data.append({
            "context": context,
            "question": question,
            "topic": "General Knowledge"
        })

    return quiz_data

squad_quiz_data = convert_squad_to_quiz(squad_dataset)


In [ ]:
print(squad_dataset["train"][0])


{'id': '5733be284776f41900661182', 'title': 'University_of_Notre_Dame', 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.', 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?', 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}


In [ ]:
def convert_mathqa_to_quiz(dataset):
    quiz_data = []

    for data in dataset["train"]:
        context = data["Problem"]
        options = data["options"].split(", ")  # Convert string options to list
        correct_answer = options[ord(data["correct"]) - ord("a")]  # Convert 'a', 'b' to index

        quiz_data.append({
            "context": context,
            "question": context,
            "options": options,
            "answer": correct_answer,
            "topic": "Mathematics"
        })

    return quiz_data

mathqa_quiz_data = convert_mathqa_to_quiz(mathqa_dataset)


In [ ]:
print(mathqa_dataset["train"][0])


{'Problem': "the banker ' s gain of a certain sum due 3 years hence at 10 % per annum is rs . 36 . what is the present worth ?", 'Rationale': '"explanation : t = 3 years r = 10 % td = ( bg × 100 ) / tr = ( 36 × 100 ) / ( 3 × 10 ) = 12 × 10 = rs . 120 td = ( pw × tr ) / 100 ⇒ 120 = ( pw × 3 × 10 ) / 100 ⇒ 1200 = pw × 3 pw = 1200 / 3 = rs . 400 answer : option a"', 'options': 'a ) rs . 400 , b ) rs . 300 , c ) rs . 500 , d ) rs . 350 , e ) none of these', 'correct': 'a', 'annotated_formula': 'divide(multiply(const_100, divide(multiply(36, const_100), multiply(3, 10))), multiply(3, 10))', 'linear_formula': 'multiply(n2,const_100)|multiply(n0,n1)|divide(#0,#1)|multiply(#2,const_100)|divide(#3,#1)|', 'category': 'gain'}


In [ ]:
def convert_ai2arc_to_quiz(dataset):
    quiz_data = []

    for data in dataset["train"]:
        question = data["question"]
        choices = data["choices"]["text"]  # Extract answer choices
        labels = data["choices"]["label"]  # Extract labels (['A', 'B', 'C', 'D'])
        answer_index = data["answerKey"]  # Correct answer (e.g., 'B')

        # Ensure answer index is valid before accessing choices
        if answer_index in labels:
            answer_text = choices[labels.index(answer_index)]  # Match label to choice
        else:
            answer_text = "Unknown"  # Fallback if answer index is invalid

        quiz_data.append({
            "context": question,  # Use the question itself as context
            "question": question,
            "options": choices,
            "answer": answer_text,  # Correct answer text
            "topic": "Science"
        })

    return quiz_data

# Convert the dataset
science_quiz_data = convert_ai2arc_to_quiz(sciq_dataset)

# Print a sample entry
print("Formatted Science Quiz Sample:", science_quiz_data[:5])


Formatted Science Quiz Sample: [{'context': 'Which factor will most likely cause a person to develop a fever?', 'question': 'Which factor will most likely cause a person to develop a fever?', 'options': ['a leg muscle relaxing after exercise', 'a bacterial population in the bloodstream', 'several viral particles on the skin', 'carbohydrates being digested in the stomach'], 'answer': 'a bacterial population in the bloodstream', 'topic': 'Science'}, {'context': 'Lichens are symbiotic organisms made of green algae and fungi. What do the green algae supply to the fungi in this symbiotic relationship?', 'question': 'Lichens are symbiotic organisms made of green algae and fungi. What do the green algae supply to the fungi in this symbiotic relationship?', 'options': ['carbon dioxide', 'food', 'protection', 'water'], 'answer': 'food', 'topic': 'Science'}, {'context': 'When a switch is used in an electrical circuit, the switch can', 'question': 'When a switch is used in an electrical circuit, 

In [ ]:
print(science_quiz_data[0])


{'context': 'Which factor will most likely cause a person to develop a fever?', 'question': 'Which factor will most likely cause a person to develop a fever?', 'options': ['a leg muscle relaxing after exercise', 'a bacterial population in the bloodstream', 'several viral particles on the skin', 'carbohydrates being digested in the stomach'], 'answer': 'a bacterial population in the bloodstream', 'topic': 'Science'}


In [ ]:
print(sciq_dataset["train"][0])


{'id': 'Mercury_7220990', 'question': 'Which factor will most likely cause a person to develop a fever?', 'choices': {'text': ['a leg muscle relaxing after exercise', 'a bacterial population in the bloodstream', 'several viral particles on the skin', 'carbohydrates being digested in the stomach'], 'label': ['A', 'B', 'C', 'D']}, 'answerKey': 'B'}


In [ ]:
print("Total Quiz Entries:", len(squad_quiz_data),len(mathqa_quiz_data), len(science_quiz_data))

Total Quiz Entries: 87599 29837 2251


In [ ]:
from datasets import Dataset

# Convert structured quiz data into a Hugging Face Dataset format
combined_data = squad_quiz_data + mathqa_quiz_data + science_quiz_data

# Create a Hugging Face Dataset
quiz_dataset = Dataset.from_list(combined_data)

# Print total dataset size
print("Total Quiz Entries:", len(quiz_dataset))


Total Quiz Entries: 119687


In [ ]:
pip install transformers datasets torch accelerate


In [ ]:
from transformers import AutoTokenizer

# Load tokenizer for T5
tokenizer = AutoTokenizer.from_pretrained("valhalla/t5-base-qg-hl")

# Tokenization function
def preprocess_function(examples):
    inputs = [f"Generate a question based on: {context}" for context in examples["context"]]
    targets = [question for question in examples["question"]]

    model_inputs = tokenizer(inputs, max_length=512, truncation=True)
    labels = tokenizer(targets, max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply tokenization to the entire dataset
tokenized_datasets = quiz_dataset.map(preprocess_function, batched=True)

# Print sample tokenized entry
print(tokenized_datasets[0])


Map:   0%|          | 0/119687 [00:00<?, ? examples/s]

{'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.', 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?', 'topic': 'General Knowledge', 'input_ids': [6939, 2206, 3, 9, 822, 3, 390, 30, 10, 30797, 120, 6, 8, 496, 65, 3, 9, 6502, 1848, 5, 71, 2916, 8, 5140, 5450, 31, 7, 2045, 22161, 19, 3, 9, 7069, 12647, 13

In [ ]:
import random

# Print 5 random samples
for i in range(10):
    sample = random.choice(tokenized_datasets)
    print(f"Topic: {sample['topic']}\nQuestion: {sample['question']}\n")


Topic: General Knowledge
Question: Ancient Galicians took control of which city in northern Portugal?

Topic: General Knowledge
Question: What feature made Wanamakers different from other stores at the time? 

Topic: General Knowledge
Question: During what decade were mechanical differential analyzers developed?

Topic: Mathematics
Question: the true discount on a bill of rs . 270 is rs . 45 . the banker ' s discount is

Topic: General Knowledge
Question: What was one of the causes of the War of the Reunions? 

Topic: General Knowledge
Question: How many major hospitals are in Detroit?

Topic: Mathematics
Question: on a trip , a cyclist averaged 8 miles per hour for the first 12 miles and 12 miles per hour for the remaining 24 miles . if the cyclist returned immediately via the same route and took a total of 7.5 hours for the round trip , what was the average speed ( in miles per hour ) for the return trip ?

Topic: Mathematics
Question: the length of a bridge in meters , which a train

In [ ]:
from datasets import DatasetDict

# Split dataset (80% train, 20% test)
split_datasets = quiz_dataset.train_test_split(test_size=0.2)

# Convert into a dataset dictionary
tokenized_datasets = DatasetDict({
    "train": split_datasets["train"],
    "test": split_datasets["test"]
})

print(tokenized_datasets)


DatasetDict({
    train: Dataset({
        features: ['context', 'question', 'topic'],
        num_rows: 95749
    })
    test: Dataset({
        features: ['context', 'question', 'topic'],
        num_rows: 23938
    })
})


In [ ]:
print("Train Dataset Size:", len(tokenized_datasets["train"]))
print("Test Dataset Size:", len(tokenized_datasets["test"]))


Train Dataset Size: 95749
Test Dataset Size: 23938


In [ ]:
from transformers import AutoTokenizer

# Load tokenizer for T5
tokenizer = AutoTokenizer.from_pretrained("valhalla/t5-base-qg-hl")

# Tokenization function with padding & truncation
def preprocess_function(examples):
    inputs = [f"Generate a question based on: {context}" for context in examples["context"]]
    targets = [question for question in examples["question"]]

    model_inputs = tokenizer(
        inputs, max_length=512, padding="max_length", truncation=True, return_tensors="pt"
    )

    labels = tokenizer(
        targets, max_length=128, padding="max_length", truncation=True, return_tensors="pt"
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply tokenization to both train and test splits
tokenized_datasets = split_datasets.map(preprocess_function, batched=True)


# Print sample tokenized data
print(tokenized_datasets["train"][0])


Map:   0%|          | 0/95749 [00:00<?, ? examples/s]

Map:   0%|          | 0/23938 [00:00<?, ? examples/s]

{'context': 'The first mention of the name "Rajasthan" appears in James Tod\'s 1829 publication Annals and Antiquities of Rajast\'han or the Central and Western Rajpoot States of India, while the earliest known record of "Rajputana" as a name for the region is in George Thomas\'s 1800 memoir Military Memories. John Keay, in his book India: A History, stated that "Rajputana" was coined by the British in 1829, John Briggs, translating Ferishta\'s history of early Islamic India, used the phrase "Rajpoot (Rajput) princes" rather than "Indian princes".', 'question': 'What is another name for the Rajasthan region?', 'topic': 'General Knowledge', 'input_ids': [6939, 2206, 3, 9, 822, 3, 390, 30, 10, 37, 166, 2652, 13, 8, 564, 96, 448, 9, 1191, 7, 6736, 121, 3475, 16, 2549, 304, 26, 31, 7, 507, 3166, 5707, 7588, 40, 7, 11, 4066, 1169, 3010, 13, 13509, 9, 7, 17, 31, 2618, 42, 8, 2808, 11, 3782, 13509, 18450, 17, 1323, 13, 1547, 6, 298, 8, 3, 16454, 801, 1368, 13, 96, 448, 9, 354, 2562, 152, 9, 1

In [ ]:
for sample in tokenized_datasets["train"].select(range(5)):
    print(f"Input Length: {len(sample['input_ids'])}, Label Length: {len(sample['labels'])}")


Input Length: 512, Label Length: 128
Input Length: 512, Label Length: 128
Input Length: 512, Label Length: 128
Input Length: 512, Label Length: 128
Input Length: 512, Label Length: 128


In [ ]:
print(tokenized_datasets)
print("Train Sample:", tokenized_datasets["train"][0])
print("Test Sample:", tokenized_datasets["test"][0])


DatasetDict({
    train: Dataset({
        features: ['context', 'question', 'topic', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 95749
    })
    test: Dataset({
        features: ['context', 'question', 'topic', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 23938
    })
})
Train Sample: {'context': 'The first mention of the name "Rajasthan" appears in James Tod\'s 1829 publication Annals and Antiquities of Rajast\'han or the Central and Western Rajpoot States of India, while the earliest known record of "Rajputana" as a name for the region is in George Thomas\'s 1800 memoir Military Memories. John Keay, in his book India: A History, stated that "Rajputana" was coined by the British in 1829, John Briggs, translating Ferishta\'s history of early Islamic India, used the phrase "Rajpoot (Rajput) princes" rather than "Indian princes".', 'question': 'What is another name for the Rajasthan region?', 'topic': 'General Knowledge', 'input_ids': [6939, 2206, 3, 9, 

In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"


In [ ]:
tokenized_datasets = tokenized_datasets.filter(lambda x: x["labels"] is not None and len(x["labels"]) > 0)


Filter:   0%|          | 0/95749 [00:00<?, ? examples/s]

Filter:   0%|          | 0/23938 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSeq2SeqLM, TrainingArguments, Trainer

# Load T5 model
model = AutoModelForSeq2SeqLM.from_pretrained("valhalla/t5-base-qg-hl")

training_args = TrainingArguments(
    output_dir="./t5-quiz-model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=4,  # Increased batch size for faster training
    per_device_eval_batch_size=4,
    learning_rate=3e-5,  # Increased LR for faster convergence
    num_train_epochs=1,  # Reduce to 1 epoch first
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=500,
    save_total_limit=2,
    gradient_accumulation_steps=1,  # ✅ Reduce for faster training
    fp16=True,  # ✅ Enable mixed precision training
    max_grad_norm=1.0
)

# ✅ Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Start training
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: nakullimbani (nakullimbani-srm-institute-of-science-and-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.139200,0.123544


TrainOutput(global_step=23938, training_loss=0.15078462595449213, metrics={'train_runtime': 9958.5186, 'train_samples_per_second': 9.615, 'train_steps_per_second': 2.404, 'total_flos': 5.830710906322944e+16, 'train_loss': 0.15078462595449213, 'epoch': 1.0})

In [ ]:
# Save model
model.save_pretrained("./t5-quiz-model")
tokenizer.save_pretrained("./t5-quiz-model")

"""# Load fine-tuned model
model = AutoModelForSeq2SeqLM.from_pretrained("./t5-quiz-model")
tokenizer = AutoTokenizer.from_pretrained("./t5-quiz-model")
"""

'# Load fine-tuned model\nmodel = AutoModelForSeq2SeqLM.from_pretrained("./t5-quiz-model")\ntokenizer = AutoTokenizer.from_pretrained("./t5-quiz-model")\n'

In [ ]:
# prompt: mount the above trained model to drive

from google.colab import drive
drive.mount('/content/drive')

# Now you can save your model to your drive
!cp -r ./t5-quiz-model /content/drive/MyDrive/


Mounted at /content/drive


In [ ]:
# prompt: download model to local machine as zip

import shutil
import os

# Define the directory containing the model files
model_directory = "./t5-quiz-model"

# Define the name of the zip file
zip_filename = "t5-quiz-model.zip"

# Create a zip archive of the model directory
shutil.make_archive(zip_filename.split('.')[0], 'zip', model_directory)

# Download the zip file
from google.colab import files
files.download(zip_filename)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!ls -lh ./t5-quiz-model/checkpoint-23938/


total 2.5G
-rw-r--r-- 1 root root 1.3K Mar 16 15:46 config.json
-rw-r--r-- 1 root root  142 Mar 16 15:46 generation_config.json
-rw-r--r-- 1 root root 851M Mar 16 15:46 model.safetensors
-rw-r--r-- 1 root root 1.7G Mar 16 15:46 optimizer.pt
-rw-r--r-- 1 root root  14K Mar 16 15:46 rng_state.pth
-rw-r--r-- 1 root root 1.1K Mar 16 15:46 scheduler.pt
-rw-r--r-- 1 root root 8.9K Mar 16 15:46 trainer_state.json
-rw-r--r-- 1 root root 5.2K Mar 16 15:46 training_args.bin


In [ ]:
def generate_mcq_question(context):
    """Generate a structured multiple-choice quiz question with four answer options using the fine-tuned T5 model."""
    input_text = f"Generate a multiple-choice question with four options based on: {context}. Format the output as: 'Question? (A) Option 1 (B) Option 2 (C) Option 3 (D) Option 4'"

    input_ids = tokenizer(input_text, return_tensors="pt").input_ids
    # Move input_ids to the same device as the model
    input_ids = input_ids.to(model.device)

    # Generate MCQ with controlled output
    output_ids = model.generate(
        input_ids,
        max_length=100,  # Slightly longer for options
        num_beams=5,  # Beam search for better quality
        no_repeat_ngram_size=2,  # Prevents repetitive sequences
        temperature=0.7,  # Adds controlled randomness
        top_k=50,
        top_p=0.95
    )

    question_with_options = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return question_with_options

# **Example Input**
context = "The Pythagorean theorem states that in a right triangle, the square of the hypotenuse is equal to the sum of the squares of the other two sides."
print(generate_mcq_question(context))

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


The Pythagorean theorem states that in a right triangle, the square of the hypotenuse is equal to what?


In [ ]:
# prompt: mount the above final fine tuned model to drive

# The model is already saved to your Google Drive in the previous code.
# This code is just for demonstration if you want to move it to another location in drive.

# Define the source and destination paths
source_path = "/content/drive/MyDrive/t5-quiz-model"  # Original location in Drive
destination_path = "/content/drive/MyDrive/new_model_location" # New location in Drive

# Use shutil.move to move the model directory
import shutil
try:
    shutil.move(source_path, destination_path)
    print(f"Model moved successfully to {destination_path}")
except FileNotFoundError:
    print(f"Error: Source directory '{source_path}' not found.")
except shutil.Error as e:
    print(f"Error moving the model: {e}")


Model moved successfully to /content/drive/MyDrive/new_model_location


In [ ]:
for sample in tokenized_datasets["train"].select(range(5)):
    if sample["labels"] is None or len(sample["labels"]) == 0:
        print("Empty Label Found:", sample)


In [ ]:
def generate_quiz_question_finetuned(context):
    """Generate a structured multiple-choice question using the fine-tuned T5 model."""
    input_text = f"Generate a question based on: {context}"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids

    # Move input_ids to the same device as the model
    input_ids = input_ids.to(model.device)

    # Generate question
    output_ids = model.generate(
        input_ids,
        max_length=50,
        do_sample=True,
        num_beams=4,  # Use beam search for more stable generation
        temperature=1.0,  # Adjust temperature to control randomness, setting it to 1.0 for default behavior
    )

    question = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return question

# Example
context = "The Pythagorean theorem states that in a right triangle, the square of the hypotenuse is equal to the sum of the squares of the other two sides."
print(generate_quiz_question_finetuned(context))

The Pythagorean theorem states that in a right triangle, the square of the hypotenuse is equal to the sum of the squares of the other two sides.


In [ ]:
# prompt: download the final model in .h5 or keras or any other format

# ... (Your existing code) ...

# Save model
model.save_pretrained("./t5-quiz-model")
tokenizer.save_pretrained("./t5-quiz-model")

# Define the directory containing the model files
model_directory = "./t5-quiz-model"

# Define the name of the zip file
zip_filename = "t5-quiz-model-1.zip"

# Create a zip archive of the model directory
shutil.make_archive(zip_filename.split('.')[0], 'zip', model_directory)

# Download the zip file
files.download(zip_filename)


In [ ]:
def generate_quiz_question_finetuned(context):
    """Generate a structured multiple-choice question using the fine-tuned T5 model."""
    input_text = f"Generate a question based on: {context}"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids

    # Move input_ids to the same device as the model
    input_ids = input_ids.to(model.device) # This line ensures input_ids are on the same device as the model

    # Generate question
    output_ids = model.generate(input_ids, max_length=50, do_sample=True)
    question = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return question

# Example
context = "The Pythagorean theorem states that in a right triangle, the square of the hypotenuse is equal to the sum of the squares of the other two sides."
print(generate_quiz_question_finetuned(context))

The Pythagorean theorem states that in a right triangle, the square of the hypotenuse is equal to the sum of the squares of which two sides?


In [ ]:
def generate_quiz_question(context):
    """Generate a well-structured multiple-choice question using T5."""
    input_text = f"Generate a properly structured multiple-choice question based on: {context}"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids

    input_ids = input_ids.to(model.device)

    # Generate question with improved decoding
    output_ids = model.generate(
        input_ids,
        max_length=50,
        do_sample=True,  # Enables diverse output
        top_k=50,  # Consider top 50 tokens
        top_p=0.95,  # Nucleus sampling
        temperature=0.7  # Adds randomness
    )

    question = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return question


In [ ]:
context = "The Pythagorean theorem states that in a right triangle, the square of the hypotenuse is equal to the sum of the squares of the other two sides."
print(generate_quiz_question(context))

What states that in a right triangle, the square of the hypotenuse is equal to the sum of the squares of the other two sides?


In [ ]:
def generate_quiz_question(context):
    """Generate a multiple-choice quiz question using the fine-tuned T5 model."""
    input_text = f"question: {context}"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids

    input_ids = input_ids.to(model.device)

    # Generate a question with better decoding parameters
    output_ids = model.generate(
        input_ids,
        max_length=50,
        num_beams=5,  # Use beam search for better quality
        no_repeat_ngram_size=2,  # Prevents repetition
        temperature=0.7,  # Adds some randomness for diversity
        top_k=50,  # Consider top 50 tokens
        top_p=0.95  # Nucleus sampling for more natural text
    )

    question = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return question

# Example
context = "The Pythagorean theorem states that in a right triangle, the square of the hypotenuse is equal to the sum of the squares of the other two sides."
print(generate_quiz_question(context))


The Pythagorean theorem states that in a right triangle, the square of the hypotenuse is equal to the sum of what?


In [ ]:
def generate_mcq_question(context):
    """Generate a multiple-choice quiz question with options using the fine-tuned T5 model."""
    input_text = f"Generate a multiple-choice question with four options based on: {context}"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids

    input_ids = input_ids.to(model.device)

    # Generate MCQ with controlled output
    output_ids = model.generate(
        input_ids,
        max_length=80,  # Slightly longer for options
        num_beams=5,  # Beam search for better quality
        no_repeat_ngram_size=2,  # Avoids repetition
        temperature=0.7,  # Adds randomness
        top_k=50,
        top_p=0.95
    )

    question_with_options = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return question_with_options

# Example
context = "The Pythagorean theorem states that in a right triangle, the square of the hypotenuse is equal to the sum of the squares of the other two sides."
print(generate_mcq_question(context))


The Pythagorean theorem states that in a right triangle, the square of the hypotenuse is equal to the sum of what?


In [ ]:
def generate_mcq_question(context):
    """Generate a structured multiple-choice quiz question with four answer options using the fine-tuned T5 model."""
    input_text = f"Generate a multiple-choice question with four options based on: {context}. Format the output as: 'Question? (A) Option 1 (B) Option 2 (C) Option 3 (D) Option 4'"

    input_ids = tokenizer(input_text, return_tensors="pt").input_ids

    input_ids = input_ids.to(model.device)

    # Generate MCQ with controlled output
    output_ids = model.generate(
        input_ids,
        max_length=100,  # Slightly longer for options
        num_beams=5,  # Beam search for better quality
        no_repeat_ngram_size=2,  # Prevents repetitive sequences
        temperature=0.7,  # Adds controlled randomness
        top_k=50,
        top_p=0.95
    )

    question_with_options = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return question_with_options

# **Example Input**
context = "The Pythagorean theorem states that in a right triangle, the square of the hypotenuse is equal to the sum of the squares of the other two sides."
print(generate_mcq_question(context))


The Pythagorean theorem states that in a right triangle, the square of the hypotenuse is equal to what?


In [ ]:
# prompt: download the final model

# Download the zip file
files.download("t5-quiz-model.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!split -b 400M /content/t5-quiz-model/checkpoint-23938/model.safetensors model_part_


In [ ]:
cat model_part_* > model.safetensors


In [ ]:
!zip model_parts.zip model_part_*


  adding: model_part_aa (deflated 7%)
  adding: model_part_ab (deflated 7%)
  adding: model_part_ac (deflated 11%)


In [ ]:
from google.colab import files
files.download("model_parts.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

# Move split files to Drive
!cp model_part_* /content/drive/MyDrive/


Mounted at /content/drive


In [ ]:
cp -rf /content/model_part_* /content/drive/MyDrive/


In [ ]:
from google.colab import files
files.download("model_part_aa")
files.download("model_part_ab")
files.download("model_part_ac")
# Continue for all parts...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!cp -r /content/model.safetensors /content/drive/MyDrive/
!cp /content/model_part_* /content/drive/MyDrive/

In [ ]:
!kill -9 -1


In [ ]:
import os
os.listdir("t5-quiz-model/checkpoint-23938")


APPROACH 2

In [ ]:
pip install transformers datasets nltk torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer
import random
import nltk
from nltk.corpus import wordnet

# Load tokenizer and model for question generation
t5_tokenizer = AutoTokenizer.from_pretrained("t5-small")
t5_model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

# Load RoBERTa for question-answering
qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2")

# Download NLTK wordnet for distractor generation
nltk.download('wordnet')


Device set to use cuda:0
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
def extract_answer(context, question):
    """Extracts the correct answer from the given passage using a QA model."""
    result = qa_pipeline({"context": context, "question": question})
    return result["answer"]


In [ ]:
def generate_distractors(answer, num_distractors=3):
    """Generates similar but incorrect options for a quiz answer."""
    synonyms = set()

    for syn in wordnet.synsets(answer):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name())

    # Remove correct answer & limit the number of distractors
    distractors = list(synonyms - {answer})[:num_distractors]

    # Fallback distractors if no synonyms found
    if len(distractors) < num_distractors:
        distractors.extend(["Option X", "Option Y", "Option Z"][:num_distractors - len(distractors)])

    return distractors


In [ ]:
def generate_distractors(answer, num_distractors=3):
    """Generates similar but incorrect options for a quiz answer."""
    synonyms = set()

    for syn in wordnet.synsets(answer):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name())

    # Remove correct answer & limit the number of distractors
    distractors = list(synonyms - {answer})[:num_distractors]

    # Fallback distractors if no synonyms found
    if len(distractors) < num_distractors:
        distractors.extend(["Option X", "Option Y", "Option Z"][:num_distractors - len(distractors)])

    return distractors


In [ ]:
def generate_mcq(context):
    """Creates a multiple-choice question with four answer choices."""

    # Generate the question
    question = generate_question(context)

    # Extract the correct answer
    correct_answer = extract_answer(context, question)

    # Generate distractors
    distractors = generate_distractors(correct_answer)

    # Combine and shuffle options
    options = [correct_answer] + distractors
    random.shuffle(options)

    # Format the MCQ
    mcq = f"Question: {question}\n"
    choices = ["A", "B", "C", "D"]

    for i, option in enumerate(options):
        mcq += f"({choices[i]}) {option}\n"

    return mcq, correct_answer


In [ ]:
# Example dataset (simulating a structured dataset)
quiz_data = [
    {"context": "The mitochondria is the powerhouse of the cell.", "topic": "Science"},
    {"context": "The sum of all angles of a triangle is 180 degrees.", "topic": "Mathematics"},
    {"context": "The capital of France is Paris.", "topic": "General Knowledge"}
]

def get_questions_by_topic(topic):
    """Filters quiz data based on selected domain."""
    return [q["context"] for q in quiz_data if q["topic"].lower() == topic.lower()]


In [ ]:
# ✅ Function 5: Run Interactive Quiz
def start_quiz():
    """Runs the interactive quiz system."""
    print("Choose a topic: Mathematics, Science, General Knowledge")
    topic = input("Enter topic: ").strip()

    questions = get_questions_by_topic(topic)

    if not questions:
        print("No questions available for this topic.")
        return

    score = 0
    total = len(questions)

    for context in questions:
        mcq, correct_answer = generate_mcq(context)
        print("\n" + mcq)

        user_answer = input("Enter your answer (A, B, C, D): ").strip().upper()
        options = ["A", "B", "C", "D"]

        if user_answer in options:
            if mcq.split(f"({user_answer}) ")[1].split("\n")[0] == correct_answer:
                print("✅ Correct!")
                score += 1
            else:
                print(f"❌ Incorrect! The correct answer was: {correct_answer}")
        else:
            print("❌ Invalid choice. Moving to next question.")

    print(f"\nFinal Score: {score}/{total}")

# ✅ Run the Quiz
start_quiz()

Choose a topic: Mathematics, Science, General Knowledge
Enter topic: Science


NameError: name 'generate_question' is not defined

In [ ]:
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer
import random
import openai

# ✅ Load models
t5_tokenizer = AutoTokenizer.from_pretrained("t5-small")
t5_model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")
qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2")

# ✅ OpenAI GPT API for better distractors (replace "YOUR-API-KEY" with actual API Key)
openai.api_key = "YOUR-API-KEY"

def generate_question(context):
    """Generates a well-structured question from context using T5."""
    input_text = f"Generate a clear quiz question based on this passage: {context}"
    input_ids = t5_tokenizer(input_text, return_tensors="pt").input_ids

    output_ids = t5_model.generate(input_ids, max_length=50, num_beams=5, no_repeat_ngram_size=2)
    question = t5_tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Ensure the question is properly formatted
    if not question.endswith("?"):
        question += "?"

    return question

def extract_answer(context, question):
    """Extracts the correct answer from context using a QA model."""
    result = qa_pipeline({"context": context, "question": question})

    answer = result["answer"]

    # Ensure the extracted answer is concise and not the full context
    if len(answer.split()) > 8:  # Limit to 8 words
        answer = " ".join(answer.split()[:8]) + "..."

    return answer

def generate_distractors_gpt(correct_answer):
    """Generates plausible wrong answer choices using GPT."""
    prompt = f"""
    Generate three incorrect but plausible multiple-choice answers to the question:
    "What is {correct_answer}?"
    The correct answer is "{correct_answer}". The incorrect answers should be related but clearly incorrect.
    """

    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=50,
        n=1,
        stop=None,
        temperature=0.7
    )

    distractors = response.choices[0].text.strip().split("\n")

    # Ensure we have exactly 3 distractors
    distractors = [d.strip() for d in distractors if d.strip()]
    while len(distractors) < 3:
        distractors.append(f"Option {chr(88 + len(distractors))}")  # "Option X", "Option Y"

    return distractors[:3]

def generate_mcq(context):
    """Creates a multiple-choice question with four answer choices."""
    question = generate_question(context)
    correct_answer = extract_answer(context, question)
    distractors = generate_distractors_gpt(correct_answer)

    # Combine and shuffle options
    options = [correct_answer] + distractors
    random.shuffle(options)

    # Format the MCQ
    mcq = f"Question: {question}\n"
    choices = ["A", "B", "C", "D"]

    for i, option in enumerate(options):
        mcq += f"({choices[i]}) {option}\n"

    return mcq, correct_answer

# ✅ Sample Quiz Data
quiz_data = [
    {"context": "The mitochondria is the powerhouse of the cell.", "topic": "Science"},
    {"context": "The sum of all angles of a triangle is 180 degrees.", "topic": "Mathematics"},
    {"context": "The capital of France is Paris.", "topic": "General Knowledge"}
]

def get_questions_by_topic(topic):
    """Filters quiz data based on selected domain."""
    return [q["context"] for q in quiz_data if q["topic"].lower() == topic.lower()]

def start_quiz():
    """Runs the interactive quiz system."""
    print("Choose a topic: Mathematics, Science, General Knowledge")
    topic = input("Enter topic: ").strip()

    questions = get_questions_by_topic(topic)

    if not questions:
        print("No questions available for this topic.")
        return

    score = 0
    total = len(questions)

    for context in questions:
        mcq, correct_answer = generate_mcq(context)
        print("\n" + mcq)

        user_answer = input("Enter your answer (A, B, C, D): ").strip().upper()
        options = ["A", "B", "C", "D"]

        if user_answer in options:
            selected_option = mcq.split(f"({user_answer}) ")[1].split("\n")[0]  # Extract selected option text
            if selected_option == correct_answer:
                print("✅ Correct!")
                score += 1
            else:
                print(f"❌ Incorrect! The correct answer was: {correct_answer}")
        else:
            print("❌ Invalid choice. Moving to next question.")

    print(f"\nFinal Score: {score}/{total}")

# ✅ Run the Quiz
start_quiz()


Device set to use cuda:0


Choose a topic: Mathematics, Science, General Knowledge
Enter topic: Science


APIRemovedInV1: 

You tried to access openai.Completion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [ ]:
# ✅ Install dependencies (Run this once)
#!pip install transformers datasets torch nltk

# ✅ Import necessary libraries
import torch
import random
import nltk
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from nltk.corpus import wordnet

# ✅ Download NLTK WordNet for generating distractors
nltk.download("wordnet")

# ✅ Load Pre-Trained T5 Model for Quiz Generation
model_name = "valhalla/t5-base-qg-hl"  # Pre-trained for question generation
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# ✅ Function to generate quiz questions from a given context
def generate_question(context):
    """Generate a quiz question from context using T5 model."""
    input_text = f"Generate a question based on: {context}"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids

    output_ids = model.generate(input_ids, max_length=50, num_beams=5)
    question = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return question

# ✅ Function to generate distractors (incorrect options) for MCQs
def generate_distractors(answer, num_distractors=3):
    """Generate distractors (incorrect answers) using WordNet."""
    synonyms = set()
    for syn in wordnet.synsets(answer):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name())

    distractors = list(synonyms - {answer})[:num_distractors]

    while len(distractors) < num_distractors:
        distractors.append(f"Option {chr(65 + len(distractors))}")  # Placeholder options

    return distractors

# ✅ Function to generate Multiple-Choice Questions (MCQs)
def generate_mcq(context):
    """Generate an MCQ with answer choices."""
    question = generate_question(context)
    answer = context.split()[0]  # Extract the first noun as the answer (simplified)

    distractors = generate_distractors(answer)
    options = distractors + [answer]
    random.shuffle(options)

    mcq = f"\nQuestion: {question}\n"
    for i, option in enumerate(options):
        mcq += f"({chr(65 + i)}) {option}\n"

    return mcq, options, answer

# ✅ Function to start the interactive quiz
def start_quiz():
    """Start an interactive quiz where the user selects a topic."""

    topics = ["Mathematics", "Science", "General Knowledge"]
    print("Choose a topic:", ", ".join(topics))

    topic = input("Enter topic: ").strip()

    if topic not in topics:
        print("❌ Invalid topic. Please choose from the list.")
        return

    # Example contexts for each topic
    topic_contexts = {
        "Mathematics": "The sum of all angles of a triangle is 180 degrees.",
        "Science": "The mitochondria is the powerhouse of the cell.",
        "General Knowledge": "Paris is the capital of France."
    }

    context = topic_contexts[topic]
    mcq, options, correct_answer = generate_mcq(context)

    print("\n", mcq)
    user_answer = input("Enter your answer (A, B, C, D): ").strip().upper()

    if options[ord(user_answer) - ord("A")] == correct_answer:
        print("✅ Correct!")
    else:
        print(f"❌ Incorrect! The correct answer was: {correct_answer}")

# ✅ Run the Quiz
start_quiz()


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


tokenizer_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/15.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

Choose a topic: Mathematics, Science, General Knowledge
Enter topic: Science

 
Question: What is the powerhouse of the cell?
(A) The
(B) Option A
(C) Option B
(D) Option C

Enter your answer (A, B, C, D): B
❌ Incorrect! The correct answer was: The


In [ ]:
# ✅ Install dependencies (Run this once)


# ✅ Import necessary libraries
import torch
import random
import nltk
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from nltk.corpus import wordnet

# ✅ Download NLTK WordNet for generating distractors
nltk.download("wordnet")

# ✅ Load Pre-Trained T5 Model for Quiz Generation
model_name = "valhalla/t5-base-qg-hl"  # Pre-trained for question generation
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# ✅ Function to extract the correct answer from the context
def extract_answer(context):
    """Extracts the most relevant answer candidate from context."""
    words = context.split()

    # Prioritize meaningful words (proper nouns, technical terms)
    for word in words:
        if word.istitle() or len(word) > 5:  # Title case & significant words
            return word

    return words[-1]  # Default to the last word if no better candidate found

# ✅ Function to generate meaningful incorrect options (distractors)
def generate_distractors(answer, num_distractors=3):
    """Generates distractors (incorrect answers) using WordNet or fallback random words."""
    synonyms = set()
    for syn in wordnet.synsets(answer):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name())

    distractors = list(synonyms - {answer})[:num_distractors]

    # If WordNet fails, create generic incorrect options
    while len(distractors) < num_distractors:
        distractors.append(f"Incorrect Option {chr(65 + len(distractors))}")

    random.shuffle(distractors)
    return distractors

# ✅ Function to generate an MCQ with correct answer + distractors
def generate_mcq(context):
    """Creates a multiple-choice question with options."""
    question = generate_question(context)
    answer = extract_answer(context)  # Extract the best answer

    distractors = generate_distractors(answer)
    options = distractors + [answer]
    random.shuffle(options)

    # Format the MCQ
    mcq = f"\nQuestion: {question}\n"
    for i, option in enumerate(options):
        mcq += f"({chr(65 + i)}) {option}\n"

    return mcq, options, answer

# ✅ Function to generate a quiz question from context
def generate_question(context):
    """Creates a quiz question from a given context using the T5 model."""
    input_text = f"Generate a question based on: {context}"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids

    output_ids = model.generate(input_ids, max_length=50, num_beams=5)
    question = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return question

# ✅ Function to start the quiz
def start_quiz():
    """Launches an interactive quiz where the user selects a topic."""

    topics = ["Mathematics", "Science", "General Knowledge"]
    print("Choose a topic:", ", ".join(topics))

    topic = input("Enter topic: ").strip()

    if topic not in topics:
        print("❌ Invalid topic. Please choose from the list.")
        return

    # Example contexts for each topic
    topic_contexts = {
        "Mathematics": "The sum of all angles of a triangle is 180 degrees.",
        "Science": "The mitochondria is the powerhouse of the cell.",
        "General Knowledge": "Paris is the capital of France."
    }

    context = topic_contexts[topic]
    mcq, options, correct_answer = generate_mcq(context)

    print("\n", mcq)
    user_answer = input("Enter your answer (A, B, C, D): ").strip().upper()

    if options[ord(user_answer) - ord("A")] == correct_answer:
        print("✅ Correct!")
    else:
        print(f"❌ Incorrect! The correct answer was: {correct_answer}")

# ✅ Run the Quiz
start_quiz()


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Choose a topic: Mathematics, Science, General Knowledge
Enter topic: General Knowledge

 
Question: What is the capital of France?
(A) capital_of_France
(B) genus_Paris
(C) French_capital
(D) Paris

Enter your answer (A, B, C, D): D
✅ Correct!


In [ ]:
"""# ✅ Install dependencies (Run this once)
!pip install transformers datasets torch nltk spacy
!python -m spacy download en_core_web_sm  # Download spaCy English model for NER"""

# ✅ Import necessary libraries
import torch
import random
import nltk
import spacy
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from nltk.corpus import wordnet

# ✅ Load spaCy model for Named Entity Recognition (NER)
nlp = spacy.load("en_core_web_sm")

# ✅ Download NLTK WordNet for generating distractors
nltk.download("wordnet")

# ✅ Load Pre-Trained T5 Model for Quiz Generation
model_name = "valhalla/t5-base-qg-hl"  # Pre-trained for question generation
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# ✅ Improved Function to Extract Meaningful Answer
def extract_answer(context):
    """Extracts the most relevant answer candidate from context using NER."""
    doc = nlp(context)
    candidates = [ent.text for ent in doc.ents]  # Extract Named Entities

    if candidates:
        return candidates[0]  # Select the most important entity

    words = context.split()

    # Prioritize meaningful words (proper nouns, technical terms)
    for word in words:
        if word.istitle() or len(word) > 5:  # Title case & significant words
            return word

    return words[-1]  # Default to the last word if no better candidate found

# ✅ Function to generate meaningful incorrect options (distractors)
def generate_distractors(answer, num_distractors=3):
    """Generates distractors (incorrect answers) using WordNet or fallback random words."""
    synonyms = set()
    for syn in wordnet.synsets(answer):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name().replace("_", " "))

    distractors = list(synonyms - {answer})[:num_distractors]

    # If WordNet fails, create generic incorrect options
    while len(distractors) < num_distractors:
        distractors.append(f"Incorrect Option {chr(65 + len(distractors))}")

    random.shuffle(distractors)
    return distractors

# ✅ Function to generate an MCQ with correct answer + distractors
def generate_mcq(context):
    """Creates a multiple-choice question with options."""
    question = generate_question(context)
    answer = extract_answer(context)  # Extract the best answer

    distractors = generate_distractors(answer)
    options = distractors + [answer]
    random.shuffle(options)

    # Format the MCQ
    mcq = f"\nQuestion: {question}\n"
    for i, option in enumerate(options):
        mcq += f"({chr(65 + i)}) {option}\n"

    return mcq, options, answer

# ✅ Function to generate a quiz question from context
def generate_question(context):
    """Creates a quiz question from a given context using the T5 model."""
    input_text = f"Generate a question based on: {context}"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids

    output_ids = model.generate(input_ids, max_length=50, num_beams=5)
    question = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return question

# ✅ Function to start the quiz
def start_quiz():
    """Launches an interactive quiz where the user selects a topic."""

    topics = ["Mathematics", "Science", "General Knowledge"]
    print("Choose a topic:", ", ".join(topics))

    topic = input("Enter topic: ").strip()

    if topic not in topics:
        print("❌ Invalid topic. Please choose from the list.")
        return

    # Example contexts for each topic
    topic_contexts = {
        "Mathematics": "The sum of all angles of a triangle is 180 degrees.",
        "Science": "The mitochondria is the powerhouse of the cell.",
        "General Knowledge": "Paris is the capital of France."
    }

    context = topic_contexts[topic]
    mcq, options, correct_answer = generate_mcq(context)

    print("\n", mcq)
    user_answer = input("Enter your answer (A, B, C, D): ").strip().upper()

    if options[ord(user_answer) - ord("A")] == correct_answer:
        print("✅ Correct!")
    else:
        print(f"❌ Incorrect! The correct answer was: {correct_answer}")

# ✅ Run the Quiz
start_quiz()


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Choose a topic: Mathematics, Science, General Knowledge
Enter topic: Science

 
Question: What is the powerhouse of the cell?
(A) Incorrect Option A
(B) Incorrect Option B
(C) Incorrect Option C
(D) The

Enter your answer (A, B, C, D): D
✅ Correct!


In [ ]:
# ✅ Import Libraries
import torch
import random
import nltk
import spacy
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from nltk.corpus import wordnet
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, util

# ✅ Load NLP Tools
nltk.download("wordnet")
nlp = spacy.load("en_core_web_sm")
bert_model = SentenceTransformer("paraphrase-MiniLM-L6-v2")  # BERT-based embeddings for distractors

# ✅ Load Pretrained T5 Model for Question Generation
model_name = "valhalla/t5-base-qg-hl"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# ✅ Load Datasets (SQuAD, MathQA, AI2-ARC)
print("Loading datasets...")
squad_dataset = load_dataset("squad")
mathqa_dataset = load_dataset("math_qa")
sciq_dataset = load_dataset("ai2_arc", "ARC-Easy")

# ✅ Convert SQuAD to Quiz Format
def convert_squad_to_quiz(dataset):
    quiz_data = []
    for data in dataset["train"]:
        quiz_data.append({
            "context": data["context"],
            "question": data["question"],
            "topic": "General Knowledge"
        })
    return quiz_data

squad_quiz_data = convert_squad_to_quiz(squad_dataset)

# ✅ Convert MathQA to Quiz Format
def convert_mathqa_to_quiz(dataset):
    quiz_data = []
    for data in dataset["train"]:
        context = data["Problem"]
        options = data["options"].split(", ")
        correct_answer = options[ord(data["correct"]) - ord("a")]

        quiz_data.append({
            "context": context,
            "question": context,
            "options": options,
            "answer": correct_answer,
            "topic": "Mathematics"
        })
    return quiz_data

mathqa_quiz_data = convert_mathqa_to_quiz(mathqa_dataset)

# ✅ Convert AI2-ARC to Quiz Format
def convert_ai2arc_to_quiz(dataset):
    quiz_data = []
    for data in dataset["train"]:
        question = data["question"]
        choices = data["choices"]["text"]
        labels = data["choices"]["label"]
        answer_index = data["answerKey"]

        if answer_index in labels:
            answer_text = choices[labels.index(answer_index)]
        else:
            answer_text = "Unknown"

        quiz_data.append({
            "context": question,
            "question": question,
            "options": choices,
            "answer": answer_text,
            "topic": "Science"
        })
    return quiz_data

science_quiz_data = convert_ai2arc_to_quiz(sciq_dataset)

# ✅ Combine All Datasets
combined_data = squad_quiz_data + mathqa_quiz_data + science_quiz_data
print("Total Quiz Entries:", len(combined_data))

# ✅ Extract the Most Relevant Answer
def extract_answer(context):
    doc = nlp(context)
    candidates = [ent.text for ent in doc.ents if len(ent.text) > 3]
    if candidates:
        return candidates[0]
    words = context.split()
    return max(words, key=len)

# ✅ Generate Distractors Using BERT Embeddings
def generate_distractors(answer, num_distractors=3):
    """Generate distractors using BERT similarity search."""
    distractors = set()

    for syn in wordnet.synsets(answer):
        for lemma in syn.lemmas():
            distractor = lemma.name().replace("_", " ")
            if distractor.lower() != answer.lower():
                distractors.add(distractor)

    if len(distractors) < num_distractors:
        all_options = ["Tree", "Mountain", "River", "Ocean", "Cloud", "Sun", "Star"]
        sentence_embeddings = bert_model.encode([answer] + all_options, convert_to_tensor=True)
        similarities = util.pytorch_cos_sim(sentence_embeddings[0], sentence_embeddings[1:])
        ranked_options = sorted(zip(all_options, similarities.tolist()), key=lambda x: x[1], reverse=True)
        for option, _ in ranked_options:
            if len(distractors) < num_distractors:
                distractors.add(option)

    return random.sample(distractors, num_distractors)

# ✅ Generate Multiple-Choice Questions
def generate_mcq(context):
    question = generate_question(context)
    answer = extract_answer(context)
    distractors = generate_distractors(answer)
    options = distractors + [answer]
    random.shuffle(options)

    mcq = f"\nQuestion: {question}\n"
    for i, option in enumerate(options):
        mcq += f"({chr(65 + i)}) {option}\n"

    return mcq, options, answer

# ✅ Generate a Quiz Question Using T5
def generate_question(context):
    input_text = f"Generate a question based on: {context}"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids
    output_ids = model.generate(input_ids, max_length=50, num_beams=5)
    question = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return question

# ✅ Start the Quiz
def start_quiz():
    topics = ["Mathematics", "Science", "General Knowledge"]
    print("Choose a topic:", ", ".join(topics))
    topic = input("Enter topic: ").strip()

    if topic not in topics:
        print("❌ Invalid topic. Please choose from the list.")
        return

    topic_data = [q for q in combined_data if q["topic"] == topic]

    if not topic_data:
        print("No questions available for this topic.")
        return

    sample = random.choice(topic_data)
    context = sample["context"]

    mcq, options, correct_answer = generate_mcq(context)
    print("\n", mcq)
    user_answer = input("Enter your answer (A, B, C, D): ").strip().upper()

    if options[ord(user_answer) - ord("A")] == correct_answer:
        print("✅ Correct!")
    else:
        print(f"❌ Incorrect! The correct answer was: {correct_answer}")

# ✅ Run the Quiz
start_quiz()


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.51k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

Generating train split:   0%|          | 0/2251 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2376 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/570 [00:00<?, ? examples/s]

Total Quiz Entries: 119687
Choose a topic: Mathematics, Science, General Knowledge
Enter topic: Science


TypeError: Population must be a sequence.  For dicts or sets, use sorted(d).

In [ ]:
!pip install transformers datasets torch nltk spacy sentence-transformers
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 93.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# ✅ Import Libraries
import torch
import random
import nltk
import spacy
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from nltk.corpus import wordnet
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, util

# ✅ Load NLP Tools
nltk.download("wordnet")
nlp = spacy.load("en_core_web_sm")
bert_model = SentenceTransformer("paraphrase-MiniLM-L6-v2")  # BERT-based embeddings for distractors


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
# ✅ Load Pretrained T5 Model for Question Generation
model_name = "valhalla/t5-base-qg-hl"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


In [ ]:
# ✅ Load Datasets (SQuAD, MathQA, AI2-ARC)
print("Loading datasets...")
squad_dataset = load_dataset("squad")
mathqa_dataset = load_dataset("math_qa")
sciq_dataset = load_dataset("ai2_arc", "ARC-Easy")


Loading datasets...


In [ ]:
# ✅ Convert SQuAD to Quiz Format
def convert_squad_to_quiz(dataset):
    quiz_data = []
    for data in dataset["train"]:
        quiz_data.append({
            "context": data["context"],
            "question": data["question"],
            "topic": "General Knowledge"
        })
    return quiz_data

squad_quiz_data = convert_squad_to_quiz(squad_dataset)

# ✅ Convert MathQA to Quiz Format
def convert_mathqa_to_quiz(dataset):
    quiz_data = []
    for data in dataset["train"]:
        context = data["Problem"]
        options = data["options"].split(", ")
        correct_answer = options[ord(data["correct"]) - ord("a")]

        quiz_data.append({
            "context": context,
            "question": context,
            "options": options,
            "answer": correct_answer,
            "topic": "Mathematics"
        })
    return quiz_data

mathqa_quiz_data = convert_mathqa_to_quiz(mathqa_dataset)

# ✅ Convert AI2-ARC to Quiz Format
def convert_ai2arc_to_quiz(dataset):
    quiz_data = []
    for data in dataset["train"]:
        question = data["question"]
        choices = data["choices"]["text"]
        labels = data["choices"]["label"]
        answer_index = data["answerKey"]

        if answer_index in labels:
            answer_text = choices[labels.index(answer_index)]
        else:
            answer_text = "Unknown"

        quiz_data.append({
            "context": question,
            "question": question,
            "options": choices,
            "answer": answer_text,
            "topic": "Science"
        })
    return quiz_data

science_quiz_data = convert_ai2arc_to_quiz(sciq_dataset)


In [ ]:
# ✅ Combine All Datasets
combined_data = squad_quiz_data + mathqa_quiz_data + science_quiz_data
print("Total Quiz Entries:", len(combined_data))


Total Quiz Entries: 119687


In [ ]:
# ✅ Extract the Most Relevant Answer
def extract_answer(context):
    doc = nlp(context)
    candidates = [ent.text for ent in doc.ents if len(ent.text) > 3]
    if candidates:
        return candidates[0]
    words = context.split()
    return max(words, key=len)


In [ ]:
# ✅ Generate Distractors Using BERT Embeddings
def generate_distractors(answer, num_distractors=3):
    """Generate distractors using BERT similarity search."""
    distractors = set()

    for syn in wordnet.synsets(answer):
        for lemma in syn.lemmas():
            distractor = lemma.name().replace("_", " ")
            if distractor.lower() != answer.lower():
                distractors.add(distractor)

    if len(distractors) < num_distractors:
        all_options = ["Tree", "Mountain", "River", "Ocean", "Cloud", "Sun", "Star"]
        sentence_embeddings = bert_model.encode([answer] + all_options, convert_to_tensor=True)
        similarities = util.pytorch_cos_sim(sentence_embeddings[0], sentence_embeddings[1:])
        ranked_options = sorted(zip(all_options, similarities.tolist()), key=lambda x: x[1], reverse=True)
        for option, _ in ranked_options:
            if len(distractors) < num_distractors:
                distractors.add(option)

    return random.sample(list(distractors), num_distractors)


In [ ]:
"""
#✅ Generate Distractors Using BERT Embeddings
def generate_distractors(answer, num_distractors=3):
    """Generate distractors using BERT similarity search."""
    distractors = set()

    for syn in wordnet.synsets(answer):
        for lemma in syn.lemmas():
            distractor = lemma.name().replace("_", " ")
            if distractor.lower() != answer.lower():
                distractors.add(distractor)

    if len(distractors) < num_distractors:
        all_options = ["Tree", "Mountain", "River", "Ocean", "Cloud", "Sun", "Star"]
        sentence_embeddings = bert_model.encode([answer] + all_options, convert_to_tensor=True)
        similarities = util.pytorch_cos_sim(sentence_embeddings[0], sentence_embeddings[1:])
        ranked_options = sorted(zip(all_options, similarities.tolist()), key=lambda x: x[1], reverse=True)
        for option, _ in ranked_options:
            if len(distractors) < num_distractors:
                distractors.add(option)

    # ✅ Ensure num_distractors is not larger than the population
    num_distractors = min(num_distractors, len(distractors))

    return random.sample(list(distractors), num_distractors)"""

SyntaxError: invalid syntax (<ipython-input-83-0dab5984d1c0>, line 4)

In [ ]:
# ✅ Generate Multiple-Choice Questions
def generate_mcq(context):
    question = generate_question(context)
    answer = extract_answer(context)
    distractors = generate_distractors(answer)
    options = distractors + [answer]
    random.shuffle(options)

    mcq = f"\nQuestion: {question}\n"
    for i, option in enumerate(options):
        mcq += f"({chr(65 + i)}) {option}\n"

    return mcq, options, answer


In [ ]:
# ✅ Generate a Quiz Question Using T5
def generate_question(context):
    input_text = f"Generate a question based on: {context}"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids
    output_ids = model.generate(input_ids, max_length=50, num_beams=5)
    question = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return question


In [ ]:
import random

# ✅ Start the Quiz (Fixes `random.choice()` issue)
def start_quiz():
    topics = ["Mathematics", "Science", "General Knowledge"]
    print("Choose a topic:", ", ".join(topics))
    topic = input("Enter topic: ").strip()

    if topic not in topics:
        print("❌ Invalid topic. Please choose from the list.")
        return

    # ✅ Convert topic_data to list explicitly
    topic_data = [q for q in combined_data if q["topic"] == topic]

    # ✅ Debugging: Check what topic_data contains
    print(f"\n🔹 Extracted {len(topic_data)} questions for topic: {topic}")

    if not topic_data:
        print("⚠ No questions available for this topic. Try another topic.")
        return

    # ✅ Ensure it's a list before using `random.choice()`
    topic_data_list = list(topic_data)
    print(f"✅ topic_data is a list? {isinstance(topic_data_list, list)}")

    # ✅ Pick a random question
    sample = random.choice(topic_data_list)

    context = sample["context"]

    mcq, options, correct_answer = generate_mcq(context)
    print("\n", mcq)

    user_answer = input("Enter your answer (A, B, C, D): ").strip().upper()

    # ✅ Convert letter (A, B, C, D) to index for correct answer
    correct_index = options.index(correct_answer)

    if user_answer == chr(ord("A") + correct_index):
        print("✅ Correct!")
    else:
        print(f"❌ Incorrect! The correct answer was: {correct_answer}")

# ✅ Run the Quiz

In [ ]:
# ✅ Run the Quiz
start_quiz()


Choose a topic: Mathematics, Science, General Knowledge
Enter topic: Science

🔹 Extracted 2251 questions for topic: Science
✅ topic_data is a list? True

 
Question: What is the most likely cause of the small pieces of rock on the trail?
(A) Pine Tree State
(B) ME
(C) Tree
(D) Maine

Enter your answer (A, B, C, D): C
❌ Incorrect! The correct answer was: Maine


APPROACH 3

In [ ]:
from datasets import load_dataset

# ✅ Load SQuAD (General Knowledge & Science)
squad_dataset = load_dataset("squad")

# ✅ Load MathQA (Mathematics)
mathqa_dataset = load_dataset("math_qa")

# ✅ Load AI2 Science Questions (Physics, Chemistry, Biology)
sciq_dataset = load_dataset("ai2_arc", "ARC-Easy")


In [ ]:
# ✅ Convert SQuAD to Quiz Format
def convert_squad_to_quiz(dataset):
    quiz_data = []
    for data in dataset["train"]:
        quiz_data.append({
            "context": data["context"],
            "question": data["question"],
            "topic": "General Knowledge"
        })
    return quiz_data

squad_quiz_data = convert_squad_to_quiz(squad_dataset)

# ✅ Convert MathQA to Quiz Format
def convert_mathqa_to_quiz(dataset):
    quiz_data = []
    for data in dataset["train"]:
        context = data["Problem"]
        options = data["options"].split(", ")
        correct_answer = options[ord(data["correct"]) - ord("a")]

        quiz_data.append({
            "context": context,
            "question": context,
            "options": options,
            "answer": correct_answer,
            "topic": "Mathematics"
        })
    return quiz_data

mathqa_quiz_data = convert_mathqa_to_quiz(mathqa_dataset)

# ✅ Convert AI2-ARC to Quiz Format
def convert_ai2arc_to_quiz(dataset):
    quiz_data = []
    for data in dataset["train"]:
        question = data["question"]
        choices = data["choices"]["text"]
        labels = data["choices"]["label"]
        answer_index = data["answerKey"]

        if answer_index in labels:
            answer_text = choices[labels.index(answer_index)]
        else:
            answer_text = "Unknown"

        quiz_data.append({
            "context": question,
            "question": question,
            "options": choices,
            "answer": answer_text,
            "topic": "Science"
        })
    return quiz_data

science_quiz_data = convert_ai2arc_to_quiz(sciq_dataset)

# ✅ Combine All Datasets
combined_data = squad_quiz_data + mathqa_quiz_data + science_quiz_data
print("Total Quiz Entries:", len(combined_data))


Total Quiz Entries: 119687


In [ ]:
from sentence_transformers import SentenceTransformer, util

# ✅ Load Sentence-BERT Model
bert_model = SentenceTransformer("paraphrase-MiniLM-L6-v2")

# ✅ Extract Answer with Proper Error Handling
def extract_answer(context):
    doc = context.split()

    # ✅ Step 1: Handle Very Short Context
    if len(doc) < 3:
        return context  # If too short, return itself

    # ✅ Step 2: Encode Words Using BERT
    try:
        embeddings = bert_model.encode(doc, convert_to_tensor=True)
        similarities = util.pytorch_cos_sim(embeddings, embeddings)

        # ✅ Step 3: Ensure Valid Similarity Selection
        max_index = similarities.argmax().item()
        if max_index >= len(doc):
            return doc[0]  # Fallback to first word if invalid index

        return doc[max_index]

    except Exception as e:
        print(f"⚠ Error in extract_answer: {e}")
        return doc[0]  # Fallback to first word in case of failure


In [ ]:
import random
from nltk.corpus import wordnet

# ✅ Generate Distractors Using BERT & WordNet
def generate_distractors(answer, num_distractors=3):
    distractors = set()

    # ✅ Step 1: Get WordNet Synonyms
    for syn in wordnet.synsets(answer):
        for lemma in syn.lemmas():
            distractor = lemma.name().replace("_", " ")
            if distractor.lower() != answer.lower():
                distractors.add(distractor)

    # ✅ Step 2: Use Sentence-BERT for Context Similarity
    all_options = ["Tree", "Mountain", "River", "Ocean", "Cloud", "Sun", "Star"]
    sentence_embeddings = bert_model.encode([answer] + all_options, convert_to_tensor=True)
    similarities = util.pytorch_cos_sim(sentence_embeddings[0], sentence_embeddings[1:])

    ranked_options = sorted(zip(all_options, similarities.tolist()), key=lambda x: x[1], reverse=True)

    for option, _ in ranked_options:
        if len(distractors) < num_distractors:
            distractors.add(option)

    # ✅ Step 3: Ensure No Duplicate or Empty Options
    distractors = list(distractors)
    if answer in distractors:
        distractors.remove(answer)

    while len(distractors) < num_distractors:
        distractors.append(f"Random Option {len(distractors)+1}")

    return random.sample(distractors, num_distractors)


In [ ]:
# ✅ Generate Multiple-Choice Questions
def generate_mcq(context):
    question = generate_question(context)
    answer = extract_answer(context)
    distractors = generate_distractors(answer)
    options = distractors + [answer]
    random.shuffle(options)

    mcq = f"\nQuestion: {question}\n"
    for i, option in enumerate(options):
        mcq += f"({chr(65 + i)}) {option}\n"

    return mcq, options, answer


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# ✅ Load T5 Model for Question Generation
tokenizer = AutoTokenizer.from_pretrained("valhalla/t5-base-qg-hl")
model = AutoModelForSeq2SeqLM.from_pretrained("valhalla/t5-base-qg-hl")

# ✅ Generate a Quiz Question Using T5
def generate_question(context):
    input_text = f"Generate a question based on: {context}"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids
    output_ids = model.generate(input_ids, max_length=50, num_beams=5)
    question = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return question


In [ ]:
import random

# ✅ Start the Quiz
def start_quiz():
    topics = ["Mathematics", "Science", "General Knowledge"]
    print("Choose a topic:", ", ".join(topics))
    topic = input("Enter topic: ").strip()

    if topic not in topics:
        print("❌ Invalid topic. Please choose from the list.")
        return

    # ✅ Convert topic_data to list explicitly
    topic_data = [q for q in combined_data if q["topic"] == topic]

    # ✅ Debugging: Check what topic_data contains
    print(f"\n🔹 Extracted {len(topic_data)} questions for topic: {topic}")

    if not topic_data:
        print("⚠ No questions available for this topic. Try another topic.")
        return

    # ✅ Pick a random question
    sample = random.choice(topic_data)
    context = sample["context"]

    mcq, options, correct_answer = generate_mcq(context)
    print("\n", mcq)

    user_answer = input("Enter your answer (A, B, C, D): ").strip().upper()
    correct_index = options.index(correct_answer)

    if user_answer == chr(ord("A") + correct_index):
        print("✅ Correct!")
    else:
        print(f"❌ Incorrect! The correct answer was: {correct_answer}")



In [ ]:
# ✅ Run the Quiz
start_quiz()


Choose a topic: Mathematics, Science, General Knowledge
Enter topic: General Knowledge

🔹 Extracted 87599 questions for topic: General Knowledge

 
Question: What does the Unorganized Borough have no government of its own?
(A) Random Option 2
(B) The
(C) Random Option 3
(D) Tree

Enter your answer (A, B, C, D): B
✅ Correct!


APPROACH 4

In [ ]:
import nltk
# Download the 'stopwords' corpus
nltk.download('stopwords')

# Now import and use stopwords
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import random
import re
from sentence_transformers import SentenceTransformer, util
from nltk.corpus import stopwords

# ✅ Load Sentence-BERT Model for Semantic Similarity
bert_model = SentenceTransformer("paraphrase-MiniLM-L6-v2")

# ✅ Load stopwords for filtering out meaningless words
stop_words = set(stopwords.words('english'))

# ✅ Final Fully Fixed extract_answer() Function
def extract_answer(context, topic):
    """Extracts the best possible answer from the given context, ensuring no failures."""

    # ✅ Extract words from context (ignore punctuation)
    words = re.findall(r'\b\w+\b', context)

    # ✅ If context is too short, return it as the answer
    if len(words) < 3:
        return context

    # ✅ Remove stopwords (avoid "which", "the", "a", etc.)
    words = [word for word in words if word.lower() not in stop_words]

    # ✅ If the word list is empty after filtering, return a fallback answer
    fallback_words = {
        "Mathematics": "Equation",
        "Science": "Energy",
        "General Knowledge": "History"
    }
    if not words:
        return fallback_words.get(topic, "Answer")  # Default fallback answer

    try:
        # ✅ Compute sentence embeddings & similarity
        embeddings = bert_model.encode(words, convert_to_tensor=True)
        similarities = util.pytorch_cos_sim(embeddings, embeddings)

        # ✅ Get the most contextually relevant word
        best_index = similarities.argmax().item()
        best_word = words[best_index]

        # ✅ Ensure it's meaningful
        if len(best_word) < 3 or best_word.lower() in ["which", "the", "it", "a"]:
            best_word = max(words, key=len)  # Use longest meaningful word instead

        return best_word

    except Exception as e:
        print(f"⚠ Error in extract_answer: {e}")
        return random.choice(words)  # ✅ Final backup to prevent failure


In [ ]:
import random
import nltk
from nltk.corpus import wordnet
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer, util

# ✅ Load Models
tokenizer = AutoTokenizer.from_pretrained("valhalla/t5-base-qg-hl")
model = AutoModelForSeq2SeqLM.from_pretrained("valhalla/t5-base-qg-hl")
bert_model = SentenceTransformer("paraphrase-MiniLM-L6-v2")

# ✅ Generate Distractors
def generate_distractors(answer, topic, num_distractors=3):
    """Generate incorrect answer choices."""
    distractors = set()

    # ✅ Generate synonyms as incorrect options
    for syn in wordnet.synsets(answer):
        for lemma in syn.lemmas():
            distractor = lemma.name().replace("_", " ")
            if distractor.lower() != answer.lower():
                distractors.add(distractor)

    # ✅ Topic-based fallback distractors
    topic_words = {
        "Mathematics": ["Algebra", "Geometry", "Equation", "Trigonometry"],
        "Science": ["Biology", "Physics", "Chemistry", "Reaction"],
        "General Knowledge": ["Country", "Capital", "History", "Politics"]
    }

    relevant_words = topic_words.get(topic, ["Option A", "Option B", "Option C", "Option D"])

    # ✅ Compute sentence embeddings for similarity
    sentence_embeddings = bert_model.encode([answer] + relevant_words, convert_to_tensor=True)
    similarities = util.pytorch_cos_sim(sentence_embeddings[0], sentence_embeddings[1:])
    ranked_options = sorted(zip(relevant_words, similarities.tolist()), key=lambda x: x[1], reverse=True)

    for option, _ in ranked_options:
        if len(distractors) < num_distractors:
            distractors.add(option)

    while len(distractors) < num_distractors:
        distractors.add(f"Random Option {len(distractors) + 1}")

    return random.sample(list(distractors), num_distractors)

# ✅ Generate Question
def generate_question(context):
    input_text = f"Generate a question based on: {context}"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids
    output_ids = model.generate(input_ids, max_length=50, num_beams=5)
    question = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return question

# ✅ Generate MCQ
def generate_mcq(context, topic):
    """Creates a multiple-choice question with correct and incorrect answer choices."""
    question = generate_question(context)
    answer = extract_answer(context, topic)
    distractors = generate_distractors(answer, topic)

    options = distractors + [answer]
    random.shuffle(options)

    mcq = f"\nQuestion: {question}\n"
    for i, option in enumerate(options):
        mcq += f"({chr(65 + i)}) {option}\n"

    return mcq, options, answer

# ✅ Start the Quiz
def start_quiz():
    topics = ["Mathematics", "Science", "General Knowledge"]
    print("Choose a topic:", ", ".join(topics))
    topic = input("Enter topic: ").strip()

    if topic not in topics:
        print("❌ Invalid topic. Please choose from the list.")
        return

    topic_data = [q for q in combined_data if q["topic"] == topic]

    if not topic_data:
        print("⚠ No questions available for this topic. Try another topic.")
        return

    sample = random.choice(topic_data)
    context = sample["context"]

    mcq, options, correct_answer = generate_mcq(context, topic)
    print("\n", mcq)

    user_answer = input("Enter your answer (A, B, C, D): ").strip().upper()
    correct_index = options.index(correct_answer)

    if user_answer == chr(ord("A") + correct_index):
        print("✅ Correct!")
    else:
        print(f"❌ Incorrect! The correct answer was: {correct_answer}")

# ✅ Run the Quiz
start_quiz()


Choose a topic: Mathematics, Science, General Knowledge
Enter topic: General Knowledge

 
Question: What is the Pli Tipitaka?
(A) Random Option 2
(B) Pāli
(C) Country
(D) Random Option 3

Enter your answer (A, B, C, D): a
❌ Incorrect! The correct answer was: Pāli


APPROACH 5

In [ ]:
from datasets import load_dataset

# ✅ Load Datasets
squad_dataset = load_dataset("squad")
mathqa_dataset = load_dataset("math_qa")
sciq_dataset = load_dataset("ai2_arc", "ARC-Easy")

# ✅ Convert SQuAD to Quiz Format
def convert_squad_to_quiz(dataset):
    quiz_data = []
    for data in dataset["train"]:
        quiz_data.append({
            "context": data["context"],
            "question": data["question"],
            "topic": "General Knowledge"
        })
    return quiz_data

squad_quiz_data = convert_squad_to_quiz(squad_dataset)

# ✅ Convert MathQA to Quiz Format
def convert_mathqa_to_quiz(dataset):
    quiz_data = []
    for data in dataset["train"]:
        context = data["Problem"]
        options = data["options"].split(", ")
        correct_answer = options[ord(data["correct"]) - ord("a")]

        quiz_data.append({
            "context": context,
            "question": context,  # Some MCQs in MathQA use context as the question
            "options": options,
            "answer": correct_answer,
            "topic": "Mathematics"
        })
    return quiz_data

mathqa_quiz_data = convert_mathqa_to_quiz(mathqa_dataset)

# ✅ Convert AI2-ARC to Quiz Format
def convert_ai2arc_to_quiz(dataset):
    quiz_data = []
    for data in dataset["train"]:
        question = data["question"]
        choices = data["choices"]["text"]
        labels = data["choices"]["label"]
        answer_index = data["answerKey"]

        if answer_index in labels:
            answer_text = choices[labels.index(answer_index)]
        else:
            answer_text = "Unknown"

        quiz_data.append({
            "context": question,
            "question": question,
            "options": choices,
            "answer": answer_text,
            "topic": "Science"
        })
    return quiz_data

science_quiz_data = convert_ai2arc_to_quiz(sciq_dataset)

# ✅ Combine All Datasets
combined_data = squad_quiz_data + mathqa_quiz_data + science_quiz_data
print("Total Quiz Entries:", len(combined_data))


Total Quiz Entries: 119687


In [ ]:
import random
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer, util

# ✅ Load Models
tokenizer = AutoTokenizer.from_pretrained("valhalla/t5-base-qg-hl")
model = AutoModelForSeq2SeqLM.from_pretrained("valhalla/t5-base-qg-hl")
bert_model = SentenceTransformer("paraphrase-MiniLM-L6-v2")

# ✅ Generate Distractors for SQuAD-style Questions
def generate_distractors(answer, topic, num_distractors=3):
    distractors = set()

    topic_words = {
        "Mathematics": ["Algebra", "Geometry", "Equation", "Trigonometry"],
        "Science": ["Biology", "Physics", "Chemistry", "Reaction"],
        "General Knowledge": ["Country", "Capital", "History", "Politics"]
    }

    relevant_words = topic_words.get(topic, ["Option A", "Option B", "Option C", "Option D"])

    # ✅ Compute sentence embeddings for similarity
    sentence_embeddings = bert_model.encode([answer] + relevant_words, convert_to_tensor=True)
    similarities = util.pytorch_cos_sim(sentence_embeddings[0], sentence_embeddings[1:])
    ranked_options = sorted(zip(relevant_words, similarities.tolist()), key=lambda x: x[1], reverse=True)

    for option, _ in ranked_options:
        if len(distractors) < num_distractors:
            distractors.add(option)

    while len(distractors) < num_distractors:
        distractors.add(f"Random Option {len(distractors) + 1}")

    return random.sample(list(distractors), num_distractors)

# ✅ Generate Question for SQuAD Data
def generate_question(context):
    input_text = f"Generate a question based on: {context}"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids
    output_ids = model.generate(input_ids, max_length=50, num_beams=5)
    question = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return question

# ✅ Generate MCQ (Use Predefined Options if Available)
def generate_mcq(sample):
    if "options" in sample and "answer" in sample:
        # ✅ Use dataset's predefined MCQ format
        options = sample["options"]
        correct_answer = sample["answer"]
        question = sample["question"]
    else:
        # ✅ Generate MCQ for SQuAD-like data
        question = generate_question(sample["context"])
        correct_answer = "Answer"  # Placeholder
        options = generate_distractors(correct_answer, sample["topic"])

    options.append(correct_answer)
    random.shuffle(options)

    mcq = f"\nQuestion: {question}\n"
    for i, option in enumerate(options):
        mcq += f"({chr(65 + i)}) {option}\n"

    return mcq, options, correct_answer


In [ ]:
# ✅ Start the Quiz
def start_quiz():
    topics = ["Mathematics", "Science", "General Knowledge"]
    print("Choose a topic:", ", ".join(topics))
    topic = input("Enter topic: ").strip()

    if topic not in topics:
        print("❌ Invalid topic. Please choose from the list.")
        return

    topic_data = [q for q in combined_data if q["topic"] == topic]

    if not topic_data:
        print("⚠ No questions available for this topic. Try another topic.")
        return

    sample = random.choice(topic_data)
    mcq, options, correct_answer = generate_mcq(sample)
    print("\n", mcq)

    user_answer = input("Enter your answer (A, B, C, D): ").strip().upper()
    correct_index = options.index(correct_answer)

    if user_answer == chr(ord("A") + correct_index):
        print("✅ Correct!")
    else:
        print(f"❌ Incorrect! The correct answer was: {correct_answer}")

# ✅ Run the Quiz
start_quiz()


Choose a topic: Mathematics, Science, General Knowledge
Enter topic: General Knowledge

 
Question: What is the mainstay of the Marshall Islands economy?
(A) Answer
(B) Random Option 3
(C) Random Option 2
(D) Country

Enter your answer (A, B, C, D): D
❌ Incorrect! The correct answer was: Answer


APPROACH 6

In [ ]:
import random
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer, util

# ✅ Load Models
tokenizer = AutoTokenizer.from_pretrained("valhalla/t5-base-qg-hl")
model = AutoModelForSeq2SeqLM.from_pretrained("valhalla/t5-base-qg-hl")
bert_model = SentenceTransformer("paraphrase-MiniLM-L6-v2")

# ✅ Extract the Correct Answer (Fixing Long Answers)
def extract_answer(context):
    """
    Extracts the most relevant short answer from the context.
    Fixes long, paragraph-style answers.
    """
    words = context.split()

    # ✅ If context is too long, return a short version
    if len(words) > 10:
        return " ".join(words[:10]) + "..."  # Shorten long answers

    return context  # Return normal short answers

# ✅ Generate Distractors for SQuAD-style Questions
def generate_distractors(answer, topic, num_distractors=3):
    distractors = set()

    topic_words = {
        "Mathematics": ["Algebra", "Geometry", "Equation", "Trigonometry"],
        "Science": ["Biology", "Physics", "Chemistry", "Reaction"],
        "General Knowledge": ["Country", "Capital", "History", "Politics"]
    }

    relevant_words = topic_words.get(topic, ["Option A", "Option B", "Option C", "Option D"])

    # ✅ Compute sentence embeddings for similarity
    sentence_embeddings = bert_model.encode([answer] + relevant_words, convert_to_tensor=True)
    similarities = util.pytorch_cos_sim(sentence_embeddings[0], sentence_embeddings[1:])
    ranked_options = sorted(zip(relevant_words, similarities.tolist()), key=lambda x: x[1], reverse=True)

    for option, _ in ranked_options:
        if len(distractors) < num_distractors:
            distractors.add(option)

    while len(distractors) < num_distractors:
        distractors.add(f"Random Option {len(distractors) + 1}")

    return random.sample(list(distractors), num_distractors)

# ✅ Generate Question for SQuAD Data
def generate_question(context):
    input_text = f"Generate a question based on: {context}"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids
    output_ids = model.generate(input_ids, max_length=50, num_beams=5)
    question = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return question

# ✅ Generate MCQ (Fixing Formatting Issues)
# ✅ Generate Multiple-Choice Questions
def generate_mcq(sample):
    if "options" in sample and "answer" in sample:
        # ✅ Use dataset's predefined MCQ format
        options = sample["options"]
        correct_answer = sample["answer"]
        question = sample["question"]

        # ✅ Fix duplicate answers issue
        options = list(set(options))  # Remove duplicates

        # ✅ Ensure exactly 4 options
        if len(options) < 4:
            distractors = generate_distractors(correct_answer, sample["topic"])
            options = distractors[:3] + [correct_answer]
        else:
            options = options[:3] + [correct_answer]

    else:
        # ✅ Generate MCQ for SQuAD-like data
        question = generate_question(sample["context"])
        correct_answer = extract_answer(sample["context"])  # ✅ Fix Long Answer Issue
        options = generate_distractors(correct_answer, sample["topic"])

    random.shuffle(options)

    # ✅ Ensure correct answer is in options
    if correct_answer not in options:
        options[random.randint(0, 3)] = correct_answer  # Replace random option

    mcq = f"\nQuestion: {question}\n"
    for i, option in enumerate(options):
        mcq += f"({chr(65 + i)}) {option}\n"

    return mcq, options, correct_answer



In [ ]:
# ✅ Start the Quiz
def start_quiz():
    topics = ["Mathematics", "Science", "General Knowledge"]
    print("Choose a topic:", ", ".join(topics))
    topic = input("Enter topic: ").strip()

    if topic not in topics:
        print("❌ Invalid topic. Please choose from the list.")
        return

    topic_data = [q for q in combined_data if q["topic"] == topic]

    if not topic_data:
        print("⚠ No questions available for this topic. Try another topic.")
        return

    sample = random.choice(topic_data)
    mcq, options, correct_answer = generate_mcq(sample)

    # ✅ Fix "Answer" problem (replace generic placeholder)
    if correct_answer == "Answer":
        correct_answer = sample.get("context", "Unknown")

    print("\n", mcq)

    user_answer = input("Enter your answer (A, B, C, D): ").strip().upper()

    # ✅ Convert letter (A, B, C, D) to index for correct answer
    correct_index = options.index(correct_answer)

    if user_answer == chr(ord("A") + correct_index):
        print("✅ Correct!")
    else:
        print(f"❌ Incorrect! The correct answer was: {correct_answer}")

# ✅ Run the Quiz
start_quiz()


Choose a topic: Mathematics, Science, General Knowledge
Enter topic: Mathematics

 
Question: the average monthly salary of laborers and supervisors in a factory is rs . 1250 per month ; where as the average monthly salary of 6 supervisors is rs . 2450 . if the average monthly salary of the laborers is rs . 950 find the number of laborers ?
(A) b ) 42 
(B) b ) 42 
(C) e ) 26
(D) c ) 78 

Enter your answer (A, B, C, D): a
✅ Correct!


APPROACH 7

In [4]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.4 MB/s eta 0:00:00


In [5]:
from datasets import load_dataset

# ✅ Load Datasets
squad_dataset = load_dataset("squad")
mathqa_dataset = load_dataset("math_qa")
sciq_dataset = load_dataset("ai2_arc", "ARC-Easy")

# ✅ Convert SQuAD to Quiz Format
def convert_squad_to_quiz(dataset):
    quiz_data = []
    for data in dataset["train"]:
        quiz_data.append({
            "context": data["context"],
            "question": data["question"],
            "topic": "General Knowledge"
        })
    return quiz_data

squad_quiz_data = convert_squad_to_quiz(squad_dataset)

# ✅ Convert MathQA to Quiz Format
def convert_mathqa_to_quiz(dataset):
    quiz_data = []
    for data in dataset["train"]:
        context = data["Problem"]
        options = data["options"].split(", ")
        correct_answer = options[ord(data["correct"]) - ord("a")]

        quiz_data.append({
            "context": context,
            "question": context,  # Some MCQs in MathQA use context as the question
            "options": options,
            "answer": correct_answer,
            "topic": "Mathematics"
        })
    return quiz_data

mathqa_quiz_data = convert_mathqa_to_quiz(mathqa_dataset)

# ✅ Convert AI2-ARC to Quiz Format
def convert_ai2arc_to_quiz(dataset):
    quiz_data = []
    for data in dataset["train"]:
        question = data["question"]
        choices = data["choices"]["text"]
        labels = data["choices"]["label"]
        answer_index = data["answerKey"]

        if answer_index in labels:
            answer_text = choices[labels.index(answer_index)]
        else:
            answer_text = "Unknown"

        quiz_data.append({
            "context": question,
            "question": question,
            "options": choices,
            "answer": answer_text,
            "topic": "Science"
        })
    return quiz_data

science_quiz_data = convert_ai2arc_to_quiz(sciq_dataset)

# ✅ Combine All Datasets
combined_data = squad_quiz_data + mathqa_quiz_data + science_quiz_data
print("Total Quiz Entries:", len(combined_data))


README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

...

validation-00000-of-00001.parquet:   0%|          | 0.00/86.1k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2251 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2376 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/570 [00:00<?, ? examples/s]

Total Quiz Entries: 119687


In [12]:
import random
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# ✅ Load T5 Model & Tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained("valhalla/t5-base-qg-hl")
tokenizer = AutoTokenizer.from_pretrained("valhalla/t5-base-qg-hl")

print("✅ Model & Tokenizer Loaded Successfully!")

# ✅ Function to Extract Clean Answer
def clean_answer_text(answer):
    """Removes unwanted prefixes (e.g., 'e ) e .', 'a ) a .') from answer choices"""
    return answer.split(") ")[-1].strip()

# ✅ Function to Generate MCQs (Using Dataset Options)
def generate_mcq(sample):
    """Generate a structured MCQ using dataset-provided options"""

    question = sample["question"]  # ✅ Directly use dataset question
    options = sample["options"]     # ✅ Directly use dataset answer choices
    correct_answer = sample["answer"]  # ✅ Use dataset-provided correct answer

    # ✅ Fix Option Formatting: Remove extra labels (e.g., "e ) e . 2.4" → "2.4")
    cleaned_options = [clean_answer_text(opt) for opt in options]

    # ✅ Ensure only 4 answer choices are included
    if len(cleaned_options) > 4:
        cleaned_options = random.sample(cleaned_options, 4)

    # ✅ Shuffle options
    random.shuffle(cleaned_options)

    # ✅ Find Correct Answer Index (AFTER Cleaning)
    correct_answer_cleaned = clean_answer_text(correct_answer)

    # ✅ Ensure correct answer exists in the 4 displayed options
    if correct_answer_cleaned not in cleaned_options:
        cleaned_options[random.randint(0, 3)] = correct_answer_cleaned

    # ✅ Format the question and options
    mcq = f"\nQuestion: {question}\n"
    for i, option in enumerate(cleaned_options):
        mcq += f"({chr(65 + i)}) {option}\n"

    return mcq, cleaned_options, correct_answer_cleaned


✅ Model & Tokenizer Loaded Successfully!


In [19]:
# ✅ Start the Quiz
def start_quiz():
    topics = ["Mathematics", "Science"]
    print("Choose a topic:", ", ".join(topics))
    topic = input("Enter topic: ").strip()

    if topic not in topics:
        print("❌ Invalid topic. Please choose from the list.")
        return

    # ✅ Extract relevant questions
    topic_data = [q for q in combined_data if q["topic"] == topic]

    if not topic_data:
        print("⚠ No questions available for this topic. Try another topic.")
        return

    # ✅ Pick a random question
    sample = random.choice(topic_data)

    mcq, options, correct_answer = generate_mcq(sample)
    print("\n", mcq)

    user_answer = input("Enter your answer (A, B, C, D): ").strip().upper()

    # ✅ Convert letter (A, B, C, D) to index for correct answer
    correct_index = options.index(correct_answer)

    if user_answer == chr(ord("A") + correct_index):
        print("✅ Correct!")
    else:
        print(f"❌ Incorrect! The correct answer was: {correct_answer}")

# ✅ Run the Quiz
start_quiz()


Choose a topic: Mathematics, Science
Enter topic: Science

 
Question: Which action causes a chemical change?
(A) leaves being crushed into pieces
(B) leaves being burned in a fire
(C) leaves dropping from a tree
(D) leaves blowing in the wind

Enter your answer (A, B, C, D): b
✅ Correct!


Print Dataset Heads

In [15]:
# Print sample questions from Mathematics dataset
print("\n🔹 Mathematics Dataset Sample:")
for i in range(5):  # Print first 5 entries
    print(f"Q{i+1}: {mathqa_quiz_data[i]['question']}")
    print(f"Options: {mathqa_quiz_data[i]['options']}")
    print(f"Correct Answer: {mathqa_quiz_data[i]['answer']}\n")

# Print sample questions from Science dataset
print("\n🔹 Science Dataset Sample:")
for i in range(5):  # Print first 5 entries
    print(f"Q{i+1}: {science_quiz_data[i]['question']}")
    print(f"Options: {science_quiz_data[i]['options']}")
    print(f"Correct Answer: {science_quiz_data[i]['answer']}\n")



🔹 Mathematics Dataset Sample:
Q1: the banker ' s gain of a certain sum due 3 years hence at 10 % per annum is rs . 36 . what is the present worth ?
Options: ['a ) rs . 400 ', 'b ) rs . 300 ', 'c ) rs . 500 ', 'd ) rs . 350 ', 'e ) none of these']
Correct Answer: a ) rs . 400 

Q2: average age of students of an adult school is 40 years . 120 new students whose average age is 32 years joined the school . as a result the average age is decreased by 4 years . find the number of students of the school after joining of the new students .
Options: ['a ) 1200 ', 'b ) 120 ', 'c ) 360 ', 'd ) 240 ', 'e ) none of these']
Correct Answer: d ) 240 

Q3: sophia finished 2 / 3 of a book . she calculated that she finished 90 more pages than she has yet to read . how long is her book ?
Options: ['a ) 229 ', 'b ) 270 ', 'c ) 877 ', 'd ) 266 ', 'e ) 281']
Correct Answer: b ) 270 

Q4: 120 is what percent of 50 ?
Options: ['a ) 5 % ', 'b ) 240 % ', 'c ) 50 % ', 'd ) 2 % ', 'e ) 500 %']
Correct Answer: b )

In [16]:
# Print dataset structure for Mathematics
print("\n🔹 Mathematics Dataset Structure:")
print(mathqa_quiz_data[0].keys())

# Print dataset structure for Science
print("\n🔹 Science Dataset Structure:")
print(science_quiz_data[0].keys())



🔹 Mathematics Dataset Structure:
dict_keys(['context', 'question', 'options', 'answer', 'topic'])

🔹 Science Dataset Structure:
dict_keys(['context', 'question', 'options', 'answer', 'topic'])


In [17]:
print("\n📊 Dataset Statistics:")
print(f"Total Mathematics Questions: {len(mathqa_quiz_data)}")
print(f"Total Science Questions: {len(science_quiz_data)}")



📊 Dataset Statistics:
Total Mathematics Questions: 29837
Total Science Questions: 2251


WORKING APP APPROACH 1

In [20]:
pip install streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.1 MB/s eta 0:00:00


In [21]:
streamlit run app.py


SyntaxError: invalid syntax (<ipython-input-21-718866ff34b9>, line 1)

MS_QUIZ_APP.PY

In [ ]:
import streamlit as st
import random
from datasets import load_dataset

# ✅ Load the datasets dynamically (Cache to prevent repeated loading)
@st.cache_data()
def load_quiz_datasets():
    # ✅ Load Mathematics Dataset
    mathqa_dataset = load_dataset("math_qa", trust_remote_code=True)

    math_questions = []
    for data in mathqa_dataset["train"]:
        context = data["Problem"]
        options = data["options"].split(", ")  # Convert options string to list
        correct_answer = options[ord(data["correct"]) - ord("a")]  # Convert 'a', 'b' -> index
        math_questions.append({
            "question": context,
            "options": options,
            "answer": correct_answer,
            "topic": "Mathematics"
        })

    # ✅ Load Science Dataset
    sciq_dataset = load_dataset("ai2_arc", "ARC-Easy")
    science_questions = []
    for data in sciq_dataset["train"]:
        question = data["question"]
        choices = data["choices"]["text"]  # Extract answer choices
        labels = data["choices"]["label"]  # Extract labels (A, B, C, D)
        answer_index = data["answerKey"]

        if answer_index in labels:
            answer_text = choices[labels.index(answer_index)]  # Match label to choice
        else:
            answer_text = "Unknown"  # Handle missing answer key properly

        science_questions.append({
            "question": question,
            "options": choices,
            "answer": answer_text,
            "topic": "Science"
        })

    return math_questions, science_questions

# ✅ Load dataset
math_questions, science_questions = load_quiz_datasets()

# ✅ Function to get a random question
def get_question(topic):
    if topic == "Mathematics":
        return random.choice(math_questions)
    elif topic == "Science":
        return random.choice(science_questions)

# ✅ Streamlit UI starts here
st.title("🧠 Interactive Quiz Generator")

# ✅ Choose subject
subject = st.selectbox("Choose a Subject:", ["Mathematics", "Science"])

# ✅ Reset question if subject is changed
if "selected_subject" not in st.session_state or st.session_state.selected_subject != subject:
    st.session_state.selected_subject = subject
    st.session_state.current_question = get_question(subject)
    st.session_state.score = 0  # Reset Score when subject changes
    st.session_state.total_attempts = 0

# ✅ Display the question
question_data = st.session_state.current_question
st.subheader(f"**Question:** {question_data['question']}")

# ✅ Display options as radio buttons
options = question_data["options"]
correct_answer = question_data["answer"]
user_choice = st.radio("Choose an option:", options, key=question_data["question"])  # Unique key for UI refresh

# ✅ Check Answer Button
if st.button("Submit Answer"):
    st.session_state.total_attempts += 1
    if user_choice == correct_answer:
        st.success("✅ Correct!")
        st.session_state.score += 1
    else:
        st.error(f"❌ Incorrect! The correct answer was: {correct_answer}")

# ✅ Display Score
st.write(f"**Score:** {st.session_state.score} / {st.session_state.total_attempts}")

# ✅ Next Question Button
if st.button("Next Question"):
    st.session_state.current_question = get_question(subject)
    st.rerun()


WORKING APP APPROACH 2

APP.PY

In [ ]:
import streamlit as st
from quiz import get_question, start_quiz  # Import the quiz function from quiz.py

# Set page title
st.set_page_config(page_title="Interactive Quiz", page_icon="🧠", layout="wide")

# Session State: Store selected subject
if "selected_subject" not in st.session_state:
    st.session_state.selected_subject = None

# Session State: Store quiz parameters
if "num_questions" not in st.session_state:
    st.session_state.num_questions = None
if "time_per_question" not in st.session_state:
    st.session_state.time_per_question = None

# 🎯 Home Page UI
def home_page():
    st.title("🎯 Welcome to the Interactive Quiz!")
    st.subheader("📚 Choose a Subject to Start Your Quiz")

    # Subject Selection Cards
    col1, col2 = st.columns(2)

    with col1:
        if st.button("📐 Mathematics", key="maths"):
            st.session_state.selected_subject = "Mathematics"
            st.session_state.num_questions = None  # Reset num_questions
            st.session_state.time_per_question = None  # Reset time per question
            st.session_state.quiz_started = False  # Ensure quiz has not started
            st.rerun()  # Redirect to parameter setup page

    with col2:
        if st.button("🔬 Science", key="science"):
            st.session_state.selected_subject = "Science"
            st.session_state.num_questions = None  # Reset num_questions
            st.session_state.time_per_question = None  # Reset time per question
            st.session_state.quiz_started = False  # Ensure quiz has not started
            st.rerun()  # Redirect to parameter setup page

# Set quiz parameters
def set_quiz_parameters():
    st.title("Set Quiz Parameters")

    st.session_state.num_questions = st.number_input("How many questions would you like?", min_value=1, max_value=20, value=10)
    st.session_state.time_per_question = st.number_input("Time per question (in seconds):", min_value=1, max_value=60, value=30)

    if st.button("Start Quiz"):
        # Initialize session state variables
        st.session_state.start_time = None
        st.session_state.score = 0
        st.session_state.total_attempts = 0
        st.session_state.streak = 0
        st.session_state.questions_asked = 0
        st.session_state.completed = False  # Track if quiz is complete
        st.session_state.quiz_started = True  # Flag quiz start
        st.session_state.current_question = get_question(st.session_state.selected_subject)
        st.rerun()

# 🚀 Run Home Page, Quiz Parameters, or Quiz
if st.session_state.selected_subject is None:
    home_page()
elif st.session_state.quiz_started is False:
    set_quiz_parameters()
else:
    start_quiz(st.session_state.selected_subject)


QUIZ.PY

In [ ]:
import streamlit as st
import random
import time
from datasets import load_dataset

# ✅ Load the datasets dynamically
def load_quiz_datasets():
    # ✅ Load Mathematics Dataset
    mathqa_dataset = load_dataset("math_qa", trust_remote_code=True)

    math_questions = []
    for data in mathqa_dataset["train"]:
        context = data["Problem"]
        options = data["options"].split(", ")  # Convert options string to list
        correct_answer = options[ord(data["correct"]) - ord("a")]  # Convert 'a', 'b' -> index

        # Handle formatting for questions like "What are the two numbers?"
        if "two numbers" in context:
            if len(options) % 2 != 0:
                options.append("N/A")  # Add a dummy value to make the number even

            formatted_options = [
                f"{options[i]} {options[i + 1]}" for i in range(0, len(options), 2)
            ]
            math_questions.append({
                "question": context,
                "options": formatted_options,
                "answer": correct_answer,
                "topic": "Mathematics"
            })
        else:
            math_questions.append({
                "question": context,
                "options": options,
                "answer": correct_answer,
                "topic": "Mathematics"
            })

    # ✅ Load Science Dataset
    sciq_dataset = load_dataset("ai2_arc", "ARC-Easy")
    science_questions = []
    for data in sciq_dataset["train"]:
        question = data["question"]
        choices = data["choices"]["text"]
        labels = data["choices"]["label"]
        answer_index = data["answerKey"]
        if answer_index in labels:
            answer_text = choices[labels.index(answer_index)]
        else:
            answer_text = "Unknown"

        science_questions.append({
            "question": question,
            "options": choices,
            "answer": answer_text,
            "topic": "Science"
        })

    return math_questions, science_questions


# Load dataset
math_questions, science_questions = load_quiz_datasets()

# ✅ Function to get a random question
def get_question(topic):
    if topic == "Mathematics":
        return random.choice(math_questions)
    elif topic == "Science":
        return random.choice(science_questions)

# ✅ Quiz Page with Timer, Progress Bar & Streak Counter
def start_quiz(subject):
    st.title("🧠 Interactive Quiz Generator")

    # Back to Home Button
    if st.button("⬅ Back to Home"):
        st.session_state.selected_subject = None
        st.session_state.quiz_started = False
        st.rerun()

    st.subheader(f"📖 Subject: {subject}")

    # Initialize session state variables
    if "current_question" not in st.session_state or "subject" not in st.session_state or st.session_state.subject != subject:
        st.session_state.current_question = get_question(subject)
        st.session_state.subject = subject
        st.session_state.score = 0
        st.session_state.total_attempts = 0
        st.session_state.streak = 0
        st.session_state.questions_asked = 0
        st.session_state.completed = False

    # Timer logic
    if st.session_state.start_time is None:
        st.session_state.start_time = time.time()

    elapsed_time = time.time() - st.session_state.start_time
    time_left = max(0, st.session_state.time_per_question - int(elapsed_time))
    st.write(f"⏳ Time left: {time_left} seconds")

    # Display streak message 🔥
    if st.session_state.streak >= 3:
        st.success(f"🔥 Streak: {st.session_state.streak} correct answers in a row! Keep going!")
    elif st.session_state.streak == 2:
        st.info(f"Good start! 2 correct answers in a row.")

    # Display question number
    question_number = st.session_state.questions_asked + 1
    st.write(f"**Question {question_number} of {st.session_state.num_questions}:** {st.session_state.current_question['question']}")

    # Options as radio buttons
    options = st.session_state.current_question["options"]
    correct_answer = st.session_state.current_question["answer"]
    user_choice = st.radio("Choose an option:", options, key="answer", disabled=st.session_state.completed)

    # ✅ Submit Answer
    if st.button("Submit Answer") and not st.session_state.completed:
        st.session_state.total_attempts += 1
        if user_choice == correct_answer:
            st.success("✅ Correct!")
            st.session_state.score += 1
            st.session_state.streak += 1  # Increase streak on correct answer
        else:
            st.error(f"❌ Incorrect! The correct answer was: {correct_answer}")
            st.session_state.streak = 0  # Reset streak on incorrect answer

        # Update progress
        st.session_state.questions_asked += 1
        st.session_state.completed = True

    # Disable answer options when time runs out
    if time_left <= 0:
        st.session_state.completed = True

    # Display Score
    st.write(f"**Score:** {st.session_state.score} / {st.session_state.total_attempts}")

    # ✅ Next Question Button
    if st.button("Next Question"):
        if st.session_state.questions_asked < st.session_state.num_questions:
            st.session_state.completed = False  # Reset for next question
            st.session_state.current_question = get_question(subject)
            st.session_state.start_time = None  # Reset the timer for next question
            st.rerun()
        else:
            # Show final score page when quiz is completed
            show_results()

def show_results():
    st.title("🎉 Quiz Completed!")
    st.subheader(f"Your final score is: {st.session_state.score} out of {st.session_state.num_questions}")
    st.subheader(f"🔥 Streak: {st.session_state.streak} correct answers in a row!")

    st.markdown("### Would you like to try again or go back to the home page?")
    if st.button("Try Again"):
        st.session_state.score = 0
        st.session_state.total_attempts = 0
        st.session_state.streak = 0
        st.session_state.questions_asked = 0
        st.session_state.completed = False
        st.session_state.start_time = None
        st.rerun()
    if st.button("Go to Home"):
        st.session_state.selected_subject = None
        st.rerun()
